# GPN-MSA Testing with Datasets

## Setup environment 

In [ ]:
# only need once for GPN-MSA use
# !pip install git+https://github.com/songlab-cal/gpn.git

In [ ]:
# !ldconfig /usr/lib64-nvidia

In [2]:
from gpn.data import GenomeMSA, Tokenizer
import gpn.model
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import torch
from transformers import AutoModel, AutoModelForMaskedLM

/cm/shared/apps/spack/0.17.3/cpu/b/opt/spack/linux-rocky8-zen/gcc-8.5.0/anaconda3-2021.05-q4munrgvh7qp4o7r3nzcdkbuph4z7375/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [3]:
from datasets import load_dataset, disable_caching
from gpn.data import load_dataset_from_file_or_dir

In [4]:
dataset_prefix = "/expanse/lustre/projects/nji102/sgriesmer/gpn/Datasets/"

In [ ]:
input_file =  dataset_prefix + "test.parquet"
dataset = load_dataset_from_file_or_dir(input_file, split="test", is_file=True)

## Create datasets

### Create Pathogenic dataset (only needed once)

In [ ]:
dataset_pathogenic = dataset.filter(lambda v: v["source"]=="ClinVar" or (v["label"]=="Common" and "missense" in v["consequence"]))
dataset_pathogenic.shape

### Create Cancer dataset (only needed once)

In [ ]:
dataset_cancer = dataset.filter(lambda v: v["source"]=="COSMIC" or (v["label"]=="Common" and "missense" in v["consequence"]))
dataset_cancer.shape

### Create Regulatory dataset (only needed once)

In [ ]:
cs = ["5_prime_UTR", "upstream_gene", "intergenic", "3_prime_UTR", "non_coding_transcript_exon"]
dataset_regulatory = dataset.filter(lambda v: v["source"]=="OMIM" or (v["label"]=="Common" and "missense" not in v["consequence"] and any([c in v["consequence"] for c in cs])))
dataset_regulatory.shape

## Save datasets as parquet files

In [ ]:
dataset_pathogenic_filename = dataset_prefix + "pathogenic.parquet"
dataset_pathogenic.to_parquet(dataset_pathogenic_filename)

In [ ]:
dataset_cancer_filename = dataset_prefix + "cancer.parquet"
dataset_cancer.to_parquet(dataset_cancer_filename)

In [ ]:
dataset_regulatory_filename = dataset_prefix + "regulatory.parquet"
dataset_regulatory.to_parquet(dataset_regulatory_filename)

# Test and Score with GPN-MSA

In [22]:
part = ""
dataset_pathogenic_filename = dataset_prefix + "PAT_dataset_XY-named-hg38" + part +".vcf"
dataset_pathogenic_df = pd.read_csv(dataset_pathogenic_filename, sep='\t')
dataset_pathogenic_filename_parquet = dataset_prefix + "PAT_dataset_XY-named-hg38.parquet"
dataset_pathogenic_df.to_parquet(dataset_pathogenic_filename_parquet)
dataset_pathogenic = load_dataset_from_file_or_dir(dataset_pathogenic_filename_parquet, split="test", is_file=True)
dataset_pathogenic.shape

(11436, 5)

In [23]:
dataset_pathogenic.features

{'chrom': Value(dtype='string', id=None),
 'pos': Value(dtype='int64', id=None),
 'name': Value(dtype='string', id=None),
 'ref': Value(dtype='string', id=None),
 'alt': Value(dtype='string', id=None)}

## Load Dataset to be Scored (if previously created)

In [ ]:
#dataset_pathogenic_filename = dataset_prefix + "pathogenic.parquet"
#dataset_pathogenic = load_dataset_from_file_or_dir(dataset_pathogenic_filename, split="test", is_file=True)
#dataset_pathogenic.shape

## Load MSA data

In [26]:
msa_path = "zip:///::https://huggingface.co/datasets/songlab/multiz100way/resolve/main/89.zarr.zip"
genome_msa = GenomeMSA(msa_path)  # can take a minute or two

Loading MSA...
Loading MSA... Done


## Load inference model

In [27]:
from gpn.msa.vep import VEPInference

model_path = "songlab/gpn-msa-sapiens"
window_size = 128
vep_inf = VEPInference(model_path, genome_msa, window_size, disable_aux_features=False)

## Pathogenic Dataset

## Subset Dataset to be Scored

In [70]:
set_start = 1000
set_end = 1300
dataset_pathogenic_set = dataset_pathogenic.select(range(set_start, set_end))
df_pathogenic_set = pd.DataFrame(dataset_pathogenic_set)
df_pathogenic_set

,chrom,pos,name,ref,alt
0,10,79946452,SNP_PAT_01001,C,T
1,10,80272484,SNP_PAT_01002,A,G
2,10,80272575,SNP_PAT_01003,C,T
3,10,80272799,SNP_PAT_01004,T,A
4,10,80273714,SNP_PAT_01005,G,A
...,...,...,...,...,...
295,11,6610130,SNP_PAT_01296,C,T
296,11,6612918,SNP_PAT_01297,C,T
297,11,6612958,SNP_PAT_01298,G,A
298,11,6613428,SNP_PAT_01299,C,G


## Tokenize Dataset

In [71]:
dataset_pathogenic_set.set_transform(vep_inf.tokenize_function)

## Set Arguments for Testing

In [72]:
from transformers import Trainer, TrainingArguments

output_dir = "/expanse/lustre/projects/nji102/sgriesmer/gpn/output"

batch_size = 100
training_args = TrainingArguments(
  output_dir=output_dir,
  per_device_eval_batch_size=batch_size,
  dataloader_num_workers=0,
  remove_unused_columns=False,
#  torch_compile=True,
#  fp16=True,
)

In [73]:
trainer = Trainer(
    model=vep_inf.model,
    args=training_args
)

## Test and Score Dataset Subset

In [74]:
# for debugging purposes
!TORCH_LOGS="+dynamo"
!TORCHDYNAMO_VERBOSE=1

In [75]:
pred=trainer.predict(test_dataset=dataset_pathogenic_set).predictions

## Add Score to Dataframe

In [76]:
gpn_score = vep_inf.postprocess(pred)
df_pathogenic_set["gpn_score"] = gpn_score
df_pathogenic_set.head()

,chrom,pos,name,ref,alt,gpn_score
0,10,79946452,SNP_PAT_01001,C,T,-1.065650
1,10,80272484,SNP_PAT_01002,A,G,-1.708707
2,10,80272575,SNP_PAT_01003,C,T,-0.865141
3,10,80272799,SNP_PAT_01004,T,A,1.565545
4,10,80273714,SNP_PAT_01005,G,A,-0.195607


## Plot shows differentiation between Common and Pathogenic mutations

In [56]:
sns.histplot(data=df_pathogenic_set, x="gpn_score", hue="label")

ValueError: Could not interpret value `label` for parameter `hue`

## Save dataframe as CSV

In [ ]:
output_file = output_dir + "/pathogenic_set_hg38_part_" + part + "_" + str(set_start) + "_" + str(set_end) + ".csv"
df_pathogenic_set.to_csv(output_file, index=False, sep=',')

## Load dataframe from CSV (if needed)

In [ ]:
output_dir = "/expanse/lustre/projects/nji102/sgriesmer/gpn/output"

input_csv_file = output_dir + "/pathogenic_set_0_39652.csv"
df_pathogenic_set = pd.read_csv(input_csv_file, sep=',')

In [ ]:
sns.histplot(data=df_pathogenic_set, x="gpn_score", hue="label")

## Calculate metrics

In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score
AUROC = roc_auc_score(df_pathogenic_set.label=="Pathogenic", -df_pathogenic_set.gpn_score)
AUPRC = average_precision_score(df_pathogenic_set.label=="Pathogenic", -df_pathogenic_set.gpn_score)
AUROC, AUPRC

## Plot ROC curve

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(df_pathogenic_set.label=="Pathogenic", -df_pathogenic_set.gpn_score)

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='b', lw=2, label=f'ROC curve (AUC = {AUROC:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

## Cancer Dataset

## Load Dataset to be Scored (if previously created)

In [ ]:
#dataset_cancer_filename = dataset_prefix + "cancer.parquet"
#dataset_cancer = load_dataset_from_file_or_dir(dataset_cancer_filename, split="test", is_file=True)
#dataset_cancer.shape

In [ ]:
part = "apaaabaaaaadac"
dataset_cancer_filename = dataset_prefix + "CAN_dataset_XY-named-hg38-part-" + part +".vcf"
dataset_cancer = load_dataset_from_file_or_dir(dataset_cancer_filename, split="test", is_file=True)
dataset_cancer.shape

In [ ]:
dataset_cancer.features

In [ ]:
dataset_cancer[0]

## Load MSA data

In [ ]:
msa_path = "zip:///::https://huggingface.co/datasets/songlab/multiz100way/resolve/main/89.zarr.zip"
genome_msa = GenomeMSA(msa_path)  # can take a minute or two

## Load inference model

In [ ]:
from gpn.msa.vep import VEPInference

model_path = "songlab/gpn-msa-sapiens"
window_size = 128
vep_inf = VEPInference(model_path, genome_msa, window_size, disable_aux_features=False)

## Subset Dataset to be Scored

In [ ]:
set_start = 0
set_end = 2
dataset_cancer_set = dataset_cancer.select(range(set_start, set_end))
df_cancer_set = pd.DataFrame(dataset_cancer_set)
df_cancer_set

## Tokenize Dataset

In [ ]:
dataset_cancer_set.set_transform(vep_inf.tokenize_function)

## Set Arguments for Testing

In [ ]:
from transformers import Trainer, TrainingArguments

output_dir = "/expanse/lustre/projects/nji102/sgriesmer/gpn/output"

batch_size = 100
training_args = TrainingArguments(
  output_dir=output_dir,
  per_device_eval_batch_size=batch_size,
  dataloader_num_workers=0,
  remove_unused_columns=False,
#  torch_compile=True,
#  fp16=True,
)

In [ ]:
trainer = Trainer(
    model=vep_inf.model,
    args=training_args
)

## Test and Score Dataset Subset

In [ ]:
# for debugging purposes
!TORCH_LOGS="+dynamo"
!TORCHDYNAMO_VERBOSE=1

In [ ]:
pred=trainer.predict(test_dataset=dataset_cancer_set).predictions

## Add Score to Dataframe

In [ ]:
gpn_score = vep_inf.postprocess(pred)
df_cancer_set["gpn_score"] = gpn_score
df_cancer_set

In [ ]:
df_cancer_set.drop([1], axis=0, inplace=True)
df_cancer_set

## Plot shows differentiation between Common and COSMIC/Frequent mutations

In [ ]:
#sns.histplot(data=df_cancer_set, x="gpn_score", hue="label")

## Save dataframe as CSV

In [ ]:
output_file = output_dir + "/cancer_set_hg38_part_" + part + "_" + str(set_start) + "_" + str(set_end) + ".csv"
df_cancer_set.to_csv(output_file, index=False, sep=',')

## Calculate metrics

In [ ]:
df_cancer_set[17585:17587]

In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score
AUROC = roc_auc_score(df_cancer_set.label=="Frequent", -gpn_score)
AUPRC = average_precision_score(df_cancer_set.label=="Frequent", -gpn_score)
AUROC, AUPRC

## Plot ROC curve

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(df_cancer_set.label=="Frequent", -gpn_score)

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='b', lw=2, label=f'ROC curve (AUC = {AUROC:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

In [ ]:
## Regulatory Dataset

## Load MSA data

In [ ]:
msa_path = "zip:///::https://huggingface.co/datasets/songlab/multiz100way/resolve/main/89.zarr.zip"
genome_msa = GenomeMSA(msa_path)  # can take a minute or two

## Load inference model

In [ ]:
from gpn.msa.vep import VEPInference

model_path = "songlab/gpn-msa-sapiens"
window_size = 128
vep_inf = VEPInference(model_path, genome_msa, window_size, disable_aux_features=False)

## Subset Dataset to be Scored

In [ ]:
output_dir = "/expanse/lustre/projects/nji102/sgriesmer/gpn/output"
results_file = output_dir + "/100-REG-results.csv"

f = open(results_file, 'a') 

for part in [ 
  "adafabad"
  ]:

  dataset_regulatory_filename = dataset_prefix + "REG_dataset_XY-named-trunc-hg38-part-" + part +".vcf"
  dataset_regulatory = load_dataset_from_file_or_dir(dataset_regulatory_filename, split="test", is_file=True)

  set_start = 0
  set_end = len(dataset_regulatory)
  dataset_regulatory_set = dataset_regulatory.select(range(set_start, set_end))
  df_regulatory_set = pd.DataFrame(dataset_regulatory_set)
  df_regulatory_set

# tokenize dataset

  dataset_regulatory_set.set_transform(vep_inf.tokenize_function)

# set arguments for testing

  from transformers import Trainer, TrainingArguments

  output_dir = "/expanse/lustre/projects/nji102/sgriesmer/gpn/output"

  batch_size = 500
  training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_eval_batch_size=batch_size,
    dataloader_num_workers=0,
    remove_unused_columns=False,
#   torch_compile=True,
#   fp16=True,
)

  trainer = Trainer(
    model=vep_inf.model,
    args=training_args
  )

# Test and score dataset subset

# for debugging purposes
  !TORCH_LOGS="+dynamo"
  !TORCHDYNAMO_VERBOSE=1

  try:
      pred=trainer.predict(test_dataset=dataset_regulatory_set).predictions
      print(part, " - yes")
      #f.write(part + ",yes")
      
      gpn_score = vep_inf.postprocess(pred)
      df_regulatory_set["gpn_score"] = gpn_score
        
      # drop stub if down to 1-result
        
      #df_regulatory_set.drop([1], axis=0, inplace=True)
    
      output_file = output_dir + "/regulatory_set_hg38_part_" + part + "_" + str(set_start) + "_" + str(set_end) + ".csv"
      df_regulatory_set.to_csv(output_file, index=False, sep=',')   
  except:
      print(part + " - no")  
      #f.write(part + ",no")
      continue

#gpn_score = vep_inf.postprocess(pred)
#df_regulatory_set["gpn_score"] = gpn_score
#df_regulatory_set.head()

# store output

#output_file = output_dir + "/regulatory_set_hg38_part_" + "_" + str(set_start) + "_" + str(set_end) + ".csv"
#df_regulatory_set.to_csv(output_file, index=False, sep=',')


## Plot shows differentiation between Common and OMIM/Pathogenic mutations

In [ ]:
sns.histplot(data=df_regulatory_set, x="gpn_score", hue="label")

## Save dataframe as CSV

In [ ]:
output_file = output_dir + "/regulatory_set" + "_" + str(set_start) + "_" + str(set_end) + ".csv"
df_regulatory_set.to_csv(output_file, index=False, sep=',')

## Load dataframe from CSV (if needed)

In [ ]:
output_dir = "/expanse/lustre/projects/nji102/sgriesmer/gpn/output"

input_csv_file = [output_dir + "/regulatory_set_0_100000.csv", 
output_dir + "/regulatory_set_100000_200000.csv",
output_dir + "/regulatory_set_200000_300000.csv",
output_dir + "/regulatory_set_300000_400000.csv",
output_dir + "/regulatory_set_400000_600000.csv",
output_dir + "/regulatory_set_600000_800000.csv"]

df_regulatory_subset = []
for f in input_csv_file:
    df_regulatory_subset.append(pd.read_csv(f, sep=','))
    
df_regulatory_set = pd.concat(df_regulatory_subset, axis=0)

In [ ]:
df_regulatory_set

In [ ]:
sns.histplot(data=df_regulatory_set, x="gpn_score", hue="label")

## Calculate metrics

In [ ]:
df_regulatory_set

In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score
AUROC = roc_auc_score(df_regulatory_set.label=="Pathogenic", -df_regulatory_set.gpn_score)
AUPRC = average_precision_score(df_regulatory_set.label=="Pathogenic", -df_regulatory_set.gpn_score)
AUROC, AUPRC

## Plot ROC curve

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(df_regulatory_set.label=="Pathogenic", -df_regulatory_set.gpn_score)

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='b', lw=2, label=f'ROC curve (AUC = {AUROC:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

## CAD Datasets

### Load GPN-MSA Model

In [5]:
msa_path = "zip:///::https://huggingface.co/datasets/songlab/multiz100way/resolve/main/89.zarr.zip"
genome_msa = GenomeMSA(msa_path)  # can take a minute or two

Loading MSA...
Loading MSA... Done


### Load Inference Model

In [6]:
from gpn.msa.vep import VEPInference

model_path = "songlab/gpn-msa-sapiens"
window_size = 128
vep_inf = VEPInference(model_path, genome_msa, window_size, disable_aux_features=False)

### Subset Dataset to be Scored

In [7]:
# keep track of successful and unsuccessful scores in results file

output_dir = "/expanse/lustre/projects/nji102/sgriesmer/gpn/output"
results_file = output_dir + "/REG_hg38_gpn_1-results.csv"

f = open(results_file, 'a') 

# create parts list

import sre_yield

parts = []
#parts = list(sre_yield.AllStrings(r'[ab]'))
parts = [
  "aaacaeaa",
  "aaacaeab",
  "aaacaeac",
  "aaacaead",
  "aaacaeae",
  "aaacaeaf",
  "aaacaeag",
  "aaacaeah",
  "aaacaeai",
  "aaacaeaj",
  "aaadabaa",
  "aaadabab",
  "aaadabac",
  "aaadabad",
  "aaadabae",
  "aaadabaf",
  "aaadabag",
  "aaadabah",
  "aaadabai",
  "aaadabaj",
  "acafagaa",
  "acafagab",
  "acafagac",
  "acafagad",
  "acafagae",
  "acafagaf",
  "acafagag",
  "acafagah",
  "acafagai",
  "acafagaj",
  "acagahaa",
  "acagahab",
  "acagahac",
  "acagahad",
  "acagahae",
  "acagahaf",
  "acagahag",
  "acagahah",
  "acagahai",
  "acagahaj",
  "acahajaa",
  "acahajab",
  "acahajac",
  "acahajad",
  "acahajae",
  "acahajaf",
  "acahajag",
  "acahajah",
  "acahajai",
  "acahajaj",
  "acajaiaa",
  "acajaiab",
  "acajaiac",
  "acajaiad",
  "acajaiae",
  "acajaiaf",
  "acajaiag",
  "acajaiah",
  "acajaiai",
  "acajaiaj",
  "adafadaa",
  "adafadab",
  "adafadac",
  "adafadad",
  "adafadae",
  "adafadaf",
  "adafadag",
  "adafadah",
  "adafadai",
  "adafadaj",
  "aeaeabaa",
  "aeaeabab",
  "aeaeabac",
  "aeaeabad",
  "aeaeabae",
  "aeaeabaf",
  "aeaeabag",
  "aeaeabah",
  "aeaeabai",
  "aeaeabaj",
  "aeaiaeaa",
  "aeaiaeab",
  "aeaiaeac",
  "aeaiaead",
  "aeaiaeae",
  "aeaiaeaf",
  "aeaiaeag",
  "aeaiaeah",
  "aeaiaeai",
  "aeaiaeaj",
  "aeaiafaa",
  "aeaiafab",
  "aeaiafac",
  "aeaiafad",
  "aeaiafae",
  "aeaiafaf",
  "aeaiafag",
  "aeaiafah",
  "aeaiafai",
  "aeaiafaj",
  "aeaiajaa",
  "aeaiajab",
  "aeaiajac",
  "aeaiajad",
  "aeaiajae",
  "aeaiajaf",
  "aeaiajag",
  "aeaiajah",
  "aeaiajai",
  "aeaiajaj",
  "afabafaa",
  "afabafab",
  "afabafac",
  "afabafad",
  "afabafae",
  "afabafaf",
  "afabafag",
  "afabafah",
  "afabafai",
  "afabafaj",
  "afacabaa",
  "afacabab",
  "afacabac",
  "afacabad",
  "afacabae",
  "afacabaf",
  "afacabag",
  "afacabah",
  "afacabai",
  "afacabaj",
  "afacaeaa",
  "afacaeab",
  "afacaeac",
  "afacaead",
  "afacaeae",
  "afacaeaf",
  "afacaeag",
  "afacaeah",
  "afacaeai",
  "afacaeaj",
  "ahababaa",
  "ahababab",
  "ahababac",
  "ahababad",
  "ahababae",
  "ahababaf",
  "ahababag",
  "ahababah",
  "ahababai",
  "ahababaj",
  "ahabacaa",
  "ahabacab",
  "ahabacac",
  "ahabacad",
  "ahabacae",
  "ahabacaf",
  "ahabacag",
  "ahabacah",
  "ahabacai",
  "ahabacaj",
  "ahabafaa",
  "ahabafab",
  "ahabafac",
  "ahabafad",
  "ahabafae",
  "ahabafaf",
  "ahabafag",
  "ahabafah",
  "ahabafai",
  "ahabafaj",
  "ahabagaa",
  "ahabagab",
  "ahabagac",
  "ahabagad",
  "ahabagae",
  "ahabagaf",
  "ahabagag",
  "ahabagah",
  "ahabagai",
  "ahabagaj",
  "ahabahaa",
  "ahabahab",
  "ahabahac",
  "ahabahad",
  "ahabahae",
  "ahabahaf",
  "ahabahag",
  "ahabahah",
  "ahabahai",
  "ahabahaj",
  "ahabaiaa",
  "ahabaiab",
  "ahabaiac",
  "ahabaiad",
  "ahabaiae",
  "ahabaiaf",
  "ahabaiag",
  "ahabaiah",
  "ahabaiai",
  "ahabaiaj",
  "ahabajaa",
  "ahabajab",
  "ahabajac",
  "ahabajad",
  "ahabajae",
  "ahabajaf",
  "ahabajag",
  "ahabajah",
  "ahabajai",
  "ahabajaj",
  "ahacaaaa",
  "ahacaaab",
  "ahacaaac",
  "ahacaaad",
  "ahacaaae",
  "ahacaaaf",
  "ahacaaag",
  "ahacaaah",
  "ahacaaai",
  "ahacaaaj",
  "ahacabaa",
  "ahacabab",
  "ahacabac",
  "ahacabad",
  "ahacabae",
  "ahacabaf",
  "ahacabag",
  "ahacabah",
  "ahacabai",
  "ahacabaj",
  "ahacadaa",
  "ahacadab",
  "ahacadac",
  "ahacadad",
  "ahacadae",
  "ahacadaf",
  "ahacadag",
  "ahacadah",
  "ahacadai",
  "ahacadaj",
  "aiabafaa",
  "aiabafab",
  "aiabafac",
  "aiabafad",
  "aiabafae",
  "aiabafaf",
  "aiabafag",
  "aiabafah",
  "aiabafai",
  "aiabafaj",
  "aiabagaa",
  "aiabagab",
  "aiabagac",
  "aiabagad",
  "aiabagae",
  "aiabagaf",
  "aiabagag",
  "aiabagah",
  "aiabagai",
  "aiabagaj",
  "aiabahaa",
  "aiabahab",
  "aiabahac",
  "aiabahad",
  "aiabahae",
  "aiabahaf",
  "aiabahag",
  "aiabahah",
  "aiabahai",
  "aiabahaj",
  "aiabaiaa",
  "aiabaiab",
  "aiabaiac",
  "aiabaiad",
  "aiabaiae",
  "aiabaiaf",
  "aiabaiag",
  "aiabaiah",
  "aiabaiai",
  "aiabaiaj",
  "aiaeajaa",
  "aiaeajab",
  "aiaeajac",
  "aiaeajad",
  "aiaeajae",
  "aiaeajaf",
  "aiaeajag",
  "aiaeajah",
  "aiaeajai",
  "aiaeajaj",
  "akaeagaa",
  "akaeagab",
  "akaeagac",
  "akaeagad",
  "akaeagae",
  "akaeagaf",
  "akaeagag",
  "akaeagah",
  "akaeagai",
  "akaeagaj",
  "asabacaa",
  "asabacab",
  "asabacac",
  "asabacad",
  "asabacae",
  "asabacaf",
  "asabacag",
  "asabacah",
  "asabacai",
  "asabacaj",
  "asabadaa",
  "asabadab",
  "asabadac",
  "asabadad",
  "asabadae",
  "asabadaf",
  "asabadag",
  "asabadah",
  "asabadai",
  "asabadaj",
  "auaaaaaa",
  "auaaaaab",
  "auaaaaac",
  "auaaaaad",
  "auaaaaae",
  "auaaaaaf",
  "auaaaaag",
  "auaaaaah",
  "auaaaaai",
  "auaaaaaj",
  "auaaabaa",
  "auaaabab",
  "auaaabac",
  "auaaabad",
  "auaaabae",
  "auaaabaf",
  "auaaabag",
  "auaaabah",
  "auaaabai",
  "auaaabaj",
  "auaaacaa",
  "auaaacab",
  "auaaacac",
  "auaaacad",
  "auaaacae",
  "auaaacaf",
  "auaaacag",
  "auaaacah",
  "auaaacai",
  "auaaacaj",
  "auaaadaa",
  "auaaadab",
  "auaaadac",
  "auaaadad",
  "auaaadae",
  "auaaadaf",
  "auaaadag",
  "auaaadah",
  "auaaadai",
  "auaaadaj",
  "auaaaeaa",
  "auaaaeab",
  "auaaaeac",
  "auaaaead",
  "auaaaeae",
  "auaaaeaf",
  "auaaaeag",
  "auaaaeah",
  "auaaaeai",
  "auaaaeaj",
  "auaaafaa",
  "auaaafab",
  "auaaafac",
  "auaaafad",
  "auaaafae",
  "auaaafaf",
  "auaaafag",
  "auaaafah",
  "auaaafai",
  "auaaafaj",
  "auaaagaa",
  "auaaagab",
  "auaaagac",
  "auaaagad",
  "auaaagae",
  "auaaagaf",
  "auaaagag",
  "auaaagah",
  "auaaagai",
  "auaaagaj",
  "auaaahaa",
  "auaaahab",
  "auaaahac",
  "auaaahad",
  "auaaahae",
  "auaaahaf",
  "auaaahag",
  "auaaahah",
  "auaaahai",
  "auaaahaj",
  "auaaaiaa",
  "auaaaiab",
  "auaaaiac",
  "auaaaiad",
  "auaaaiae",
  "auaaaiaf",
  "auaaaiag",
  "auaaaiah",
  "auaaaiai",
  "auaaaiaj",
  "auaaajaa",
  "auaaajab",
  "auaaajac",
  "auaaajad",
  "auaaajae",
  "auaaajaf",
  "auaaajag",
  "auaaajah",
  "auaaajai",
  "auaaajaj",
  "auabaaaa",
  "auabaaab",
  "auabaaac",
  "auabaaad",
  "auabaaae",
  "auabaaaf",
  "auabaaag",
  "auabaaah",
  "auabaaai",
  "auabaaaj",
  "auababaa",
  "auababab",
  "auababac",
  "auababad",
  "auababae",
  "auababaf",
  "auababag",
  "auababah",
  "auababai",
  "auababaj",
  "auabacaa",
  "auabacab",
  "auabacac",
  "auabacad",
  "auabacae",
  "auabacaf",
  "auabacag",
  "auabacah",
  "auabacai",
  "auabacaj",
  "auabadaa",
  "auabadab",
  "auabadac",
  "auabadad",
  "auabadae",
  "auabadaf",
  "auabadag",
  "auabadah",
  "auabadai",
  "auabadaj",
  "auabaeaa",
  "auabaeab",
  "auabaeac",
  "auabaead",
  "auabaeae",
  "auabaeaf",
  "auabaeag",
  "auabaeah",
  "auabaeai",
  "auabaeaj",
  "auabafaa",
  "auabafab",
  "auabafac",
  "auabafad",
  "auabafae",
  "auabafaf",
  "auabafag",
  "auabafah",
  "auabafai",
  "auabafaj",
  "auabagaa",
  "auabagab",
  "auabagac",
  "auabagad",
  "auabagae",
  "auabagaf",
  "auabagag",
  "auabagah",
  "auabagai",
  "auabagaj",
  "auabahaa",
  "auabahab",
  "auabahac",
  "auabahad",
  "auabahae",
  "auabahaf",
  "auabahag",
  "auabahah",
  "auabahai",
  "auabahaj",
  "auabaiaa",
  "auabaiab",
  "auabaiac",
  "auabaiad",
  "auabaiae",
  "auabaiaf",
  "auabaiag",
  "auabaiah",
  "auabaiai",
  "auabaiaj",
  "auabajaa",
  "auabajab",
  "auabajac",
  "auabajad",
  "auabajae",
  "auabajaf",
  "auabajag",
  "auabajah",
  "auabajai",
  "auabajaj",
  "auacaaaa",
  "auacaaab",
  "auacaaac",
  "auacaaad",
  "auacaaae",
  "auacaaaf",
  "auacaaag",
  "auacaaah",
  "auacaaai",
  "auacaaaj",
  "auacabaa",
  "auacabab",
  "auacabac",
  "auacabad",
  "auacabae",
  "auacabaf",
  "auacabag",
  "auacabah",
  "auacabai",
  "auacabaj",
  "auacacaa",
  "auacacab",
  "auacacac",
  "auacacad",
  "auacacae",
  "auacacaf",
  "auacacag",
  "auacacah",
  "auacacai",
  "auacacaj",
  "auacadaa",
  "auacadab",
  "auacadac",
  "auacadad",
  "auacadae",
  "auacadaf",
  "auacadag",
  "auacadah",
  "auacadai",
  "auacadaj",
  "auacaeaa",
  "auacaeab",
  "auacaeac",
  "auacaead",
  "auacaeae",
  "auacaeaf",
  "auacaeag",
  "auacaeah",
  "auacaeai",
  "auacaeaj",
  "auacafaa",
  "auacafab",
  "auacafac",
  "auacafad",
  "auacafae",
  "auacafaf",
  "auacafag",
  "auacafah",
  "auacafai",
  "auacafaj",
  "auacagaa",
  "auacagab",
  "auacagac",
  "auacagad",
  "auacagae",
  "auacagaf",
  "auacagag",
  "auacagah",
  "auacagai",
  "auacagaj",
  "auacahaa",
  "auacahab",
  "auacahac",
  "auacahad",
  "auacahae",
  "auacahaf",
  "auacahag",
  "auacahah",
  "auacahai",
  "auacahaj",
  "auacaiaa",
  "auacaiab",
  "auacaiac",
  "auacaiad",
  "auacaiae",
  "auacaiaf",
  "auacaiag",
  "auacaiah",
  "auacaiai",
  "auacaiaj",
  "auacajaa",
  "auacajab",
  "auacajac",
  "auacajad",
  "auacajae",
  "auacajaf",
  "auacajag",
  "auacajah",
  "auacajai",
  "auacajaj",
  "auadaaaa",
  "auadaaab",
  "auadaaac",
  "auadaaad",
  "auadaaae",
  "auadaaaf",
  "auadaaag",
  "auadaaah",
  "auadaaai",
  "auadaaaj",
  "auadabaa",
  "auadabab",
  "auadabac",
  "auadabad",
  "auadabae",
  "auadabaf",
  "auadabag",
  "auadabah",
  "auadabai",
  "auadabaj",
  "auadacaa",
  "auadacab",
  "auadacac",
  "auadacad",
  "auadacae",
  "auadacaf",
  "auadacag",
  "auadacah",
  "auadacai",
  "auadacaj",
  "auadadaa",
  "auadadab",
  "auadadac",
  "auadadad",
  "auadadae",
  "auadadaf",
  "auadadag",
  "auadadah",
  "auadadai",
  "auadadaj",
  "auadaeaa",
  "auadaeab",
  "auadaeac",
  "auadaead",
  "auadaeae",
  "auadaeaf",
  "auadaeag",
  "auadaeah",
  "auadaeai",
  "auadaeaj",
  "auadafaa",
  "auadafab",
  "auadafac",
  "auadafad",
  "auadafae",
  "auadafaf",
  "auadafag",
  "auadafah",
  "auadafai",
  "auadafaj",
  "auadagaa",
  "auadagab",
  "auadagac",
  "auadagad",
  "auadagae",
  "auadagaf",
  "auadagag",
  "auadagah",
  "auadagai",
  "auadagaj",
  "auadahaa",
  "auadahab",
  "auadahac",
  "auadahad",
  "auadahae",
  "auadahaf",
  "auadahag",
  "auadahah",
  "auadahai",
  "auadahaj",
  "auadaiaa",
  "auadaiab",
  "auadaiac",
  "auadaiad",
  "auadaiae",
  "auadaiaf",
  "auadaiag",
  "auadaiah",
  "auadaiai",
  "auadaiaj",
  "auadajaa",
  "auadajab",
  "auadajac",
  "auadajad",
  "auadajae",
  "auadajaf",
  "auadajag",
  "auadajah",
  "auadajai",
  "auadajaj",
  "auaeaaaa",
  "auaeaaab",
  "auaeaaac",
  "auaeaaad",
  "auaeaaae",
  "auaeaaaf",
  "auaeaaag",
  "auaeaaah",
  "auaeaaai",
  "auaeaaaj",
  "auaeabaa",
  "auaeabab",
  "auaeabac",
  "auaeabad",
  "auaeabae",
  "auaeabaf",
  "auaeabag",
  "auaeabah",
  "auaeabai",
  "auaeabaj",
  "auaeacaa",
  "auaeacab",
  "auaeacac",
  "auaeacad",
  "auaeacae",
  "auaeacaf",
  "auaeacag",
  "auaeacah",
  "auaeacai",
  "auaeacaj",
  "auaeadaa",
  "auaeadab",
  "auaeadac",
  "auaeadad",
  "auaeadae",
  "auaeadaf",
  "auaeadag",
  "auaeadah",
  "auaeadai",
  "auaeadaj",
  "auaeaeaa",
  "auaeaeab",
  "auaeaeac",
  "auaeaead",
  "auaeaeae",
  "auaeaeaf",
  "auaeaeag",
  "auaeaeah",
  "auaeaeai",
  "auaeaeaj",
  "auaeafaa",
  "auaeafab",
  "auaeafac",
  "auaeafad",
  "auaeafae",
  "auaeafaf",
  "auaeafag",
  "auaeafah",
  "auaeafai",
  "auaeafaj",
  "auaeagaa",
  "auaeagab",
  "auaeagac",
  "auaeagad",
  "auaeagae",
  "auaeagaf",
  "auaeagag",
  "auaeagah",
  "auaeagai",
  "auaeagaj",
  "auaeahaa",
  "auaeahab",
  "auaeahac",
  "auaeahad",
  "auaeahae",
  "auaeahaf",
  "auaeahag",
  "auaeahah",
  "auaeahai",
  "auaeahaj",
  "auaeaiaa",
  "auaeaiab",
  "auaeaiac",
  "auaeaiad",
  "auaeaiae",
  "auaeaiaf",
  "auaeaiag",
  "auaeaiah",
  "auaeaiai",
  "auaeaiaj",
  "auaeajaa",
  "auaeajab",
  "auaeajac",
  "auaeajad",
  "auaeajae",
  "auaeajaf",
  "auaeajag",
  "auaeajah",
  "auaeajai",
  "auaeajaj",
  "auafaaaa",
  "auafaaab",
  "auafaaac",
  "auafaaad",
  "auafaaae",
  "auafaaaf",
  "auafaaag",
  "auafaaah",
  "auafaaai",
  "auafaaaj",
  "auafabaa",
  "auafabab",
  "auafabac",
  "auafabad",
  "auafabae",
  "auafabaf",
  "auafabag",
  "auafabah",
  "auafabai",
  "auafabaj",
  "auafacaa",
  "auafacab",
  "auafacac",
  "auafacad",
  "auafacae",
  "auafacaf",
  "auafacag",
  "auafacah",
  "auafacai",
  "auafacaj",
  "auafadaa",
  "auafadab",
  "auafadac",
  "auafadad",
  "auafadae",
  "auafadaf",
  "auafadag",
  "auafadah",
  "auafadai",
  "auafadaj",
  "auafaeaa",
  "auafaeab",
  "auafaeac",
  "auafaead",
  "auafaeae",
  "auafaeaf",
  "auafaeag",
  "auafaeah",
  "auafaeai",
  "auafaeaj",
  "auafafaa",
  "auafafab",
  "auafafac",
  "auafafad",
  "auafafae",
  "auafafaf",
  "auafafag",
  "auafafah",
  "auafafai",
  "auafafaj",
  "auafagaa",
  "auafagab",
  "auafagac",
  "auafagad",
  "auafagae",
  "auafagaf",
  "auafagag",
  "auafagah",
  "auafagai",
  "auafagaj",
  "auafahaa",
  "auafahab",
  "auafahac",
  "auafahad",
  "auafahae",
  "auafahaf",
  "auafahag",
  "auafahah",
  "auafahai",
  "auafahaj",
  "auafaiaa",
  "auafaiab",
  "auafaiac",
  "auafaiad",
  "auafaiae",
  "auafaiaf",
  "auafaiag",
  "auafaiah",
  "auafaiai",
  "auafaiaj",
  "auafajaa",
  "auafajab",
  "auafajac",
  "auafajad",
  "auafajae",
  "auafajaf",
  "auafajag",
  "auafajah",
  "auafajai",
  "auafajaj",
  "auagaaaa",
  "auagaaab",
  "auagaaac",
  "auagaaad",
  "auagaaae",
  "auagaaaf",
  "auagaaag",
  "auagaaah",
  "auagaaai",
  "auagaaaj",
  "auagabaa",
  "auagabab",
  "auagabac",
  "auagabad",
  "auagabae",
  "auagabaf",
  "auagabag",
  "auagabah",
  "auagabai",
  "auagabaj",
  "auagacaa",
  "auagacab",
  "auagacac",
  "auagacad",
  "auagacae",
  "auagacaf",
  "auagacag",
  "auagacah",
  "auagacai",
  "auagacaj",
  "auagadaa",
  "auagadab",
  "auagadac",
  "auagadad",
  "auagadae",
  "auagadaf",
  "auagadag",
  "auagadah",
  "auagadai",
  "auagadaj",
  "auagaeaa",
  "auagaeab",
  "auagaeac",
  "auagaead",
  "auagaeae",
  "auagaeaf",
  "auagaeag",
  "auagaeah",
  "auagaeai",
  "auagaeaj",
  "auagafaa",
  "auagafab",
  "auagafac",
  "auagafad",
  "auagafae",
  "auagafaf",
  "auagafag",
  "auagafah",
  "auagafai",
  "auagafaj",
  "auagagaa",
  "auagagab",
  "auagagac",
  "auagagad",
  "auagagae",
  "auagagaf",
  "auagagag",
  "auagagah",
  "auagagai",
  "auagagaj",
  "auagahaa",
  "auagahab",
  "auagahac",
  "auagahad",
  "auagahae",
  "auagahaf",
  "auagahag",
  "auagahah",
  "auagahai",
  "auagahaj",
  "auagaiaa",
  "auagaiab",
  "auagaiac",
  "auagaiad",
  "auagaiae",
  "auagaiaf",
  "auagaiag",
  "auagaiah",
  "auagaiai",
  "auagaiaj",
  "auagajaa",
  "auagajab",
  "auagajac",
  "auagajad",
  "auagajae",
  "auagajaf",
  "auagajag",
  "auagajah",
  "auagajai",
  "auagajaj",
  "auahaaaa",
  "auahaaab",
  "auahaaac",
  "auahaaad",
  "auahaaae",
  "auahaaaf",
  "auahaaag",
  "auahaaah",
  "auahaaai",
  "auahaaaj",
  "auahabaa",
  "auahabab",
  "auahabac",
  "auahabad",
  "auahabae",
  "auahabaf",
  "auahabag",
  "auahabah",
  "auahabai",
  "auahabaj",
  "auahacaa",
  "auahacab",
  "auahacac",
  "auahacad",
  "auahacae",
  "auahacaf",
  "auahacag",
  "auahacah",
  "auahacai",
  "auahacaj",
  "auahadaa",
  "auahadab",
  "auahadac",
  "auahadad",
  "auahadae",
  "auahadaf",
  "auahadag",
  "auahadah",
  "auahadai",
  "auahadaj",
  "auahaeaa",
  "auahaeab",
  "auahaeac",
  "auahaead",
  "auahaeae",
  "auahaeaf",
  "auahaeag",
  "auahaeah",
  "auahaeai",
  "auahaeaj",
  "auahafaa",
  "auahafab",
  "auahafac",
  "auahafad",
  "auahafae",
  "auahafaf",
  "auahafag",
  "auahafah",
  "auahafai",
  "auahafaj",
  "auahagaa",
  "auahagab",
  "auahagac",
  "auahagad",
  "auahagae",
  "auahagaf",
  "auahagag",
  "auahagah",
  "auahagai",
  "auahagaj",
  "auahahaa",
  "auahahab",
  "auahahac",
  "auahahad",
  "auahahae",
  "auahahaf",
  "auahahag",
  "auahahah",
  "auahahai",
  "auahahaj",
  "auahaiaa",
  "auahaiab",
  "auahaiac",
  "auahaiad",
  "auahaiae",
  "auahaiaf",
  "auahaiag",
  "auahaiah",
  "auahaiai",
  "auahaiaj",
  "auahajaa",
  "auahajab",
  "auahajac",
  "auahajad",
  "auahajae",
  "auahajaf",
  "auahajag",
  "auahajah",
  "auahajai",
  "auahajaj",
  "auaiaaaa",
  "auaiaaab",
  "auaiaaac",
  "auaiaaad",
  "auaiaaae",
  "auaiaaaf",
  "auaiaaag",
  "auaiaaah",
  "auaiaaai",
  "auaiaaaj",
  "auaiabaa",
  "auaiabab",
  "auaiabac",
  "auaiabad",
  "auaiabae",
  "auaiabaf",
  "auaiabag",
  "auaiabah",
  "auaiabai",
  "auaiabaj",
  "auaiacaa",
  "auaiacab",
  "auaiacac",
  "auaiacad",
  "auaiacae",
  "auaiacaf",
  "auaiacag",
  "auaiacah",
  "auaiacai",
  "auaiacaj",
  "auaiadaa",
  "auaiadab",
  "auaiadac",
  "auaiadad",
  "auaiadae",
  "auaiadaf",
  "auaiadag",
  "auaiadah",
  "auaiadai",
  "auaiadaj",
  "auaiaeaa",
  "auaiaeab",
  "auaiaeac",
  "auaiaead",
  "auaiaeae",
  "auaiaeaf",
  "auaiaeag",
  "auaiaeah",
  "auaiaeai",
  "auaiaeaj",
  "auaiafaa",
  "auaiafab",
  "auaiafac",
  "auaiafad",
  "auaiafae",
  "auaiafaf",
  "auaiafag",
  "auaiafah",
  "auaiafai",
  "auaiafaj",
  "auaiagaa",
  "auaiagab",
  "auaiagac",
  "auaiagad",
  "auaiagae",
  "auaiagaf",
  "auaiagag",
  "auaiagah",
  "auaiagai",
  "auaiagaj",
  "auaiahaa",
  "auaiahab",
  "auaiahac",
  "auaiahad",
  "auaiahae",
  "auaiahaf",
  "auaiahag",
  "auaiahah",
  "auaiahai",
  "auaiahaj",
  "auaiaiaa",
  "auaiaiab",
  "auaiaiac",
  "auaiaiad",
  "auaiaiae",
  "auaiaiaf",
  "auaiaiag",
  "auaiaiah",
  "auaiaiai",
  "auaiaiaj",
  "auaiajaa",
  "auaiajab",
  "auaiajac",
  "auaiajad",
  "auaiajae",
  "auaiajaf",
  "auaiajag",
  "auaiajah",
  "auaiajai",
  "auaiajaj",
  "auajaaaa",
  "auajaaab",
  "auajaaac",
  "auajaaad",
  "auajaaae",
  "auajaaaf",
  "auajaaag",
  "auajaaah",
  "auajaaai",
  "auajaaaj",
  "auajabaa",
  "auajabab",
  "auajabac",
  "auajabad",
  "auajabae",
  "auajabaf",
  "auajabag",
  "auajabah",
  "auajabai",
  "auajabaj",
  "auajacaa",
  "auajacab",
  "auajacac",
  "auajacad",
  "auajacae",
  "auajacaf",
  "auajacag",
  "auajacah",
  "auajacai",
  "auajacaj",
  "auajadaa",
  "auajadab",
  "auajadac",
  "auajadad",
  "auajadae",
  "auajadaf",
  "auajadag",
  "auajadah",
  "auajadai",
  "auajadaj",
  "auajaeaa",
  "auajaeab",
  "auajaeac",
  "auajaead",
  "auajaeae",
  "auajaeaf",
  "auajaeag",
  "auajaeah",
  "auajaeai",
  "auajaeaj",
  "auajafaa",
  "auajafab",
  "auajafac",
  "auajafad",
  "auajafae",
  "auajafaf",
  "auajafag",
  "auajafah",
  "auajafai",
  "auajafaj",
  "auajagaa",
  "auajagab",
  "auajagac",
  "auajagad",
  "auajagae",
  "auajagaf",
  "auajagag",
  "auajagah",
  "auajagai",
  "auajagaj",
  "auajahaa",
  "auajahab",
  "auajahac",
  "auajahad",
  "auajahae",
  "auajahaf",
  "auajahag",
  "auajahah",
  "auajahai",
  "auajahaj",
  "auajaiaa",
  "auajaiab",
  "auajaiac",
  "auajaiad",
  "auajaiae",
  "auajaiaf",
  "auajaiag",
  "auajaiah",
  "auajaiai",
  "auajaiaj",
  "auajajaa",
  "auajajab",
  "auajajac",
  "auajajad",
  "auajajae",
  "auajajaf",
  "auajajag",
  "auajajah",
  "auajajai",
  "auajajaj",
  "avaaaaaa",
  "avaaaaab",
  "avaaaaac",
  "avaaaaad",
  "avaaaaae",
  "avaaaaaf",
  "avaaaaag",
  "avaaaaah",
  "avaaaaai",
  "avaaaaaj",
  "avaaabaa",
  "avaaabab",
  "avaaabac",
  "avaaabad",
  "avaaabae",
  "avaaabaf",
  "avaaabag",
  "avaaabah",
  "avaaabai",
  "avaaabaj",
  "avaaacaa",
  "avaaacab",
  "avaaacac",
  "avaaacad",
  "avaaacae",
  "avaaacaf",
  "avaaacag",
  "avaaacah",
  "avaaacai",
  "avaaacaj",
  "avaaadaa",
  "avaaadab",
  "avaaadac",
  "avaaadad",
  "avaaadae",
  "avaaadaf",
  "avaaadag",
  "avaaadah",
  "avaaadai",
  "avaaadaj",
  "avaaaeaa",
  "avaaaeab",
  "avaaaeac",
  "avaaaead",
  "avaaaeae",
  "avaaaeaf",
  "avaaaeag",
  "avaaaeah",
  "avaaaeai",
  "avaaaeaj",
  "avaaafaa",
  "avaaafab",
  "avaaafac",
  "avaaafad",
  "avaaafae",
  "avaaafaf",
  "avaaafag",
  "avaaafah",
  "avaaafai",
  "avaaafaj",
  "avaaagaa",
  "avaaagab",
  "avaaagac",
  "avaaagad",
  "avaaagae",
  "avaaagaf",
  "avaaagag",
  "avaaagah",
  "avaaagai",
  "avaaagaj",
  "avaaahaa",
  "avaaahab",
  "avaaahac",
  "avaaahad",
  "avaaahae",
  "avaaahaf",
  "avaaahag",
  "avaaahah",
  "avaaahai",
  "avaaahaj",
  "avaaaiaa",
  "avaaaiab",
  "avaaaiac",
  "avaaaiad",
  "avaaaiae",
  "avaaaiaf",
  "avaaaiag",
  "avaaaiah",
  "avaaaiai",
  "avaaaiaj",
  "avaaajaa",
  "avaaajab",
  "avaaajac",
  "avaaajad",
  "avaaajae",
  "avaaajaf",
  "avaaajag",
  "avaaajah",
  "avaaajai",
  "avaaajaj",
  "avabaaaa",
  "avabaaab",
  "avabaaac",
  "avabaaad",
  "avabaaae",
  "avabaaaf",
  "avabaaag",
  "avabaaah",
  "avabaaai",
  "avabaaaj",
  "avababaa",
  "avababab",
  "avababac",
  "avababad",
  "avababae",
  "avababaf",
  "avababag",
  "avababah",
  "avababai",
  "avababaj",
  "avabacaa",
  "avabacab",
  "avabacac",
  "avabacad",
  "avabacae",
  "avabacaf",
  "avabacag",
  "avabacah",
  "avabacai",
  "avabacaj",
  "avabadaa",
  "avabadab",
  "avabadac",
  "avabadad",
  "avabadae",
  "avabadaf",
  "avabadag",
  "avabadah",
  "avabadai",
  "avabadaj",
  "avabaeaa",
  "avabaeab",
  "avabaeac",
  "avabaead",
  "avabaeae",
  "avabaeaf",
  "avabaeag",
  "avabaeah",
  "avabaeai",
  "avabaeaj",
  "avabafaa",
  "avabafab",
  "avabafac",
  "avabafad",
  "avabafae",
  "avabafaf",
  "avabafag",
  "avabafah",
  "avabafai",
  "avabafaj",
  "avabagaa",
  "avabagab",
  "avabagac",
  "avabagad",
  "avabagae",
  "avabagaf",
  "avabagag",
  "avabagah",
  "avabagai",
  "avabagaj",
  "avabahaa",
  "avabahab",
  "avabahac",
  "avabahad",
  "avabahae",
  "avabahaf",
  "avabahag",
  "avabahah",
  "avabahai",
  "avabahaj",
  "avabaiaa",
  "avabaiab",
  "avabaiac",
  "avabaiad",
  "avabaiae",
  "avabaiaf",
  "avabaiag",
  "avabaiah",
  "avabaiai",
  "avabaiaj",
  "avabajaa",
  "avabajab",
  "avabajac",
  "avabajad",
  "avabajae",
  "avabajaf",
  "avabajag",
  "avabajah",
  "avabajai",
  "avabajaj",
  "avacaaaa",
  "avacaaab",
  "avacaaac",
  "avacaaad",
  "avacaaae",
  "avacaaaf",
  "avacaaag",
  "avacaaah",
  "avacaaai",
  "avacaaaj",
  "avacabaa",
  "avacabab",
  "avacabac",
  "avacabad",
  "avacabae",
  "avacabaf",
  "avacabag",
  "avacabah",
  "avacabai",
  "avacabaj",
  "avacacaa",
  "avacacab",
  "avacacac",
  "avacacad",
  "avacacae",
  "avacacaf",
  "avacacag",
  "avacacah",
  "avacacai",
  "avacacaj",
  "avacadaa",
  "avacadab",
  "avacadac",
  "avacadad",
  "avacadae",
  "avacadaf",
  "avacadag",
  "avacadah",
  "avacadai",
  "avacadaj",
  "avacaeaa",
  "avacaeab",
  "avacaeac",
  "avacaead",
  "avacaeae",
  "avacaeaf",
  "avacaeag",
  "avacaeah",
  "avacaeai",
  "avacaeaj",
  "avacafaa",
  "avacafab",
  "avacafac",
  "avacafad",
  "avacafae",
  "avacafaf",
  "avacafag",
  "avacafah",
  "avacafai",
  "avacafaj",
  "avacagaa",
  "avacagab",
  "avacagac",
  "avacagad",
  "avacagae",
  "avacagaf",
  "avacagag",
  "avacagah",
  "avacagai",
  "avacagaj",
  "avacahaa",
  "avacahab",
  "avacahac",
  "avacahad",
  "avacahae",
  "avacahaf",
  "avacahag",
  "avacahah",
  "avacahai",
  "avacahaj",
  "avacaiaa",
  "avacaiab",
  "avacaiac",
  "avacaiad",
  "avacaiae",
  "avacaiaf",
  "avacaiag",
  "avacaiah",
  "avacaiai",
  "avacaiaj",
  "avacajaa",
  "avacajab",
  "avacajac",
  "avacajad",
  "avacajae",
  "avacajaf",
  "avacajag",
  "avacajah",
  "avacajai",
  "avacajaj",
  "avadaaaa",
  "avadaaab",
  "avadaaac",
  "avadaaad",
  "avadaaae",
  "avadaaaf",
  "avadaaag",
  "avadaaah",
  "avadaaai",
  "avadaaaj",
  "avadabaa",
  "avadabab",
  "avadabac",
  "avadabad",
  "avadabae",
  "avadabaf",
  "avadabag",
  "avadabah",
  "avadabai",
  "avadabaj",
  "avadacaa",
  "avadacab",
  "avadacac",
  "avadacad",
  "avadacae",
  "avadacaf",
  "avadacag",
  "avadacah",
  "avadacai",
  "avadacaj",
  "avadadaa",
  "avadadab",
  "avadadac",
  "avadadad",
  "avadadae",
  "avadadaf",
  "avadadag",
  "avadadah",
  "avadadai",
  "avadadaj",
  "avadaeaa",
  "avadaeab",
  "avadaeac",
  "avadaead",
  "avadaeae",
  "avadaeaf",
  "avadaeag",
  "avadaeah",
  "avadaeai",
  "avadaeaj",
  "avadafaa",
  "avadafab",
  "avadafac",
  "avadafad",
  "avadafae",
  "avadafaf",
  "avadafag",
  "avadafah",
  "avadafai",
  "avadafaj",
  "avadagaa",
  "avadagab",
  "avadagac",
  "avadagad",
  "avadagae",
  "avadagaf",
  "avadagag",
  "avadagah",
  "avadagai",
  "avadagaj",
  "avadahaa",
  "avadahab",
  "avadahac",
  "avadahad",
  "avadahae",
  "avadahaf",
  "avadahag",
  "avadahah",
  "avadahai",
  "avadahaj",
  "avadaiaa",
  "avadaiab",
  "avadaiac",
  "avadaiad",
  "avadaiae",
  "avadaiaf",
  "avadaiag",
  "avadaiah",
  "avadaiai",
  "avadaiaj",
  "avadajaa",
  "avadajab",
  "avadajac",
  "avadajad",
  "avadajae",
  "avadajaf",
  "avadajag",
  "avadajah",
  "avadajai",
  "avadajaj",
  "avaeaaaa",
  "avaeaaab",
  "avaeaaac",
  "avaeaaad",
  "avaeaaae",
  "avaeaaaf",
  "avaeaaag",
  "avaeaaah",
  "avaeaaai",
  "avaeaaaj",
  "avaeabaa",
  "avaeabab",
  "avaeabac",
  "avaeabad",
  "avaeabae",
  "avaeabaf",
  "avaeabag",
  "avaeabah",
  "avaeabai",
  "avaeabaj",
  "avaeacaa",
  "avaeacab",
  "avaeacac",
  "avaeacad",
  "avaeacae",
  "avaeacaf",
  "avaeacag",
  "avaeacah",
  "avaeacai",
  "avaeacaj",
  "avaeadaa",
  "avaeadab",
  "avaeadac",
  "avaeadad",
  "avaeadae",
  "avaeadaf",
  "avaeadag",
  "avaeadah",
  "avaeadai",
  "avaeadaj",
  "avaeaeaa",
  "avaeaeab",
  "avaeaeac",
  "avaeaead",
  "avaeaeae",
  "avaeaeaf",
  "avaeaeag",
  "avaeaeah",
  "avaeaeai",
  "avaeaeaj",
  "avaeafaa",
  "avaeafab",
  "avaeafac",
  "avaeafad",
  "avaeafae",
  "avaeafaf",
  "avaeafag",
  "avaeafah",
  "avaeafai",
  "avaeafaj",
  "avaeagaa",
  "avaeagab",
  "avaeagac",
  "avaeagad",
  "avaeagae",
  "avaeagaf",
  "avaeagag",
  "avaeagah",
  "avaeagai",
  "avaeagaj",
  "avaeahaa",
  "avaeahab",
  "avaeahac",
  "avaeahad",
  "avaeahae",
  "avaeahaf",
  "avaeahag",
  "avaeahah",
  "avaeahai",
  "avaeahaj",
  "avaeaiaa",
  "avaeaiab",
  "avaeaiac",
  "avaeaiad",
  "avaeaiae",
  "avaeaiaf",
  "avaeaiag",
  "avaeaiah",
  "avaeaiai",
  "avaeaiaj",
  "avaeajaa",
  "avaeajab",
  "avaeajac",
  "avaeajad",
  "avaeajae",
  "avaeajaf",
  "avaeajag",
  "avaeajah",
  "avaeajai",
  "avaeajaj",
  "avafaaaa",
  "avafaaab",
  "avafaaac",
  "avafaaad",
  "avafaaae",
  "avafaaaf",
  "avafaaag",
  "avafaaah",
  "avafaaai",
  "avafaaaj",
  "avafabaa",
  "avafabab",
  "avafabac",
  "avafabad",
  "avafabae",
  "avafabaf",
  "avafabag",
  "avafabah",
  "avafabai",
  "avafabaj",
  "avafacaa",
  "avafacab",
  "avafacac",
  "avafacad",
  "avafacae",
  "avafacaf",
  "avafacag",
  "avafacah",
  "avafacai",
  "avafacaj",
  "avafadaa",
  "avafadab",
  "avafadac",
  "avafadad",
  "avafadae",
  "avafadaf",
  "avafadag",
  "avafadah",
  "avafadai",
  "avafadaj",
  "avafaeaa",
  "avafaeab",
  "avafaeac",
  "avafaead",
  "avafaeae",
  "avafaeaf",
  "avafaeag",
  "avafaeah",
  "avafaeai",
  "avafaeaj",
  "avafafaa",
  "avafafab",
  "avafafac",
  "avafafad",
  "avafafae",
  "avafafaf",
  "avafafag",
  "avafafah",
  "avafafai",
  "avafafaj",
  "avafagaa",
  "avafagab",
  "avafagac",
  "avafagad",
  "avafagae",
  "avafagaf",
  "avafagag",
  "avafagah",
  "avafagai",
  "avafagaj",
  "avafahaa",
  "avafahab",
  "avafahac",
  "avafahad",
  "avafahae",
  "avafahaf",
  "avafahag",
  "avafahah",
  "avafahai",
  "avafahaj",
  "avafaiaa",
  "avafaiab",
  "avafaiac",
  "avafaiad",
  "avafaiae",
  "avafaiaf",
  "avafaiag",
  "avafaiah",
  "avafaiai",
  "avafaiaj",
  "avafajaa",
  "avafajab",
  "avafajac",
  "avafajad",
  "avafajae",
  "avafajaf",
  "avafajag",
  "avafajah",
  "avafajai",
  "avafajaj",
  "avagaaaa",
  "avagaaab",
  "avagaaac",
  "avagaaad",
  "avagaaae",
  "avagaaaf",
  "avagaaag",
  "avagaaah",
  "avagaaai",
  "avagaaaj",
  "avagabaa",
  "avagabab",
  "avagabac",
  "avagabad",
  "avagabae",
  "avagabaf",
  "avagabag",
  "avagabah",
  "avagabai",
  "avagabaj",
  "avagacaa",
  "avagacab",
  "avagacac",
  "avagacad",
  "avagacae",
  "avagacaf",
  "avagacag",
  "avagacah",
  "avagacai",
  "avagacaj",
  "avagadaa",
  "avagadab",
  "avagadac",
  "avagadad",
  "avagadae",
  "avagadaf",
  "avagadag",
  "avagadah",
  "avagadai",
  "avagadaj",
  "avagaeaa",
  "avagaeab",
  "avagaeac",
  "avagaead",
  "avagaeae",
  "avagaeaf",
  "avagaeag",
  "avagaeah",
  "avagaeai",
  "avagaeaj",
  "avagafaa",
  "avagafab",
  "avagafac",
  "avagafad",
  "avagafae",
  "avagafaf",
  "avagafag",
  "avagafah",
  "avagafai",
  "avagafaj",
  "avagagaa",
  "avagagab",
  "avagagac",
  "avagagad",
  "avagagae",
  "avagagaf",
  "avagagag",
  "avagagah",
  "avagagai",
  "avagagaj",
  "avagahaa",
  "avagahab",
  "avagahac",
  "avagahad",
  "avagahae",
  "avagahaf",
  "avagahag",
  "avagahah",
  "avagahai",
  "avagahaj",
  "avagaiaa",
  "avagaiab",
  "avagaiac",
  "avagaiad",
  "avagaiae",
  "avagaiaf",
  "avagaiag",
  "avagaiah",
  "avagaiai",
  "avagaiaj",
  "avagajaa",
  "avagajab",
  "avagajac",
  "avagajad",
  "avagajae",
  "avagajaf",
  "avagajag",
  "avagajah",
  "avagajai",
  "avagajaj",
  "avahaaaa",
  "avahaaab",
  "avahaaac",
  "avahaaad",
  "avahaaae",
  "avahaaaf",
  "avahaaag",
  "avahaaah",
  "avahaaai",
  "avahaaaj",
  "avahabaa",
  "avahabab",
  "avahabac",
  "avahabad",
  "avahabae",
  "avahabaf",
  "avahabag",
  "avahabah",
  "avahabai",
  "avahabaj",
  "avahacaa",
  "avahacab",
  "avahacac",
  "avahacad",
  "avahacae",
  "avahacaf",
  "avahacag",
  "avahacah",
  "avahacai",
  "avahacaj",
  "avahadaa",
  "avahadab",
  "avahadac",
  "avahadad",
  "avahadae",
  "avahadaf",
  "avahadag",
  "avahadah",
  "avahadai",
  "avahadaj",
  "avahaeaa",
  "avahaeab",
  "avahaeac",
  "avahaead",
  "avahaeae",
  "avahaeaf",
  "avahaeag",
  "avahaeah",
  "avahaeai",
  "avahaeaj",
  "avahafaa",
  "avahafab",
  "avahafac",
  "avahafad",
  "avahafae",
  "avahafaf",
  "avahafag",
  "avahafah",
  "avahafai",
  "avahafaj",
  "avahagaa",
  "avahagab",
  "avahagac",
  "avahagad",
  "avahagae",
  "avahagaf",
  "avahagag",
  "avahagah",
  "avahagai",
  "avahagaj",
  "avahahaa",
  "avahahab",
  "avahahac",
  "avahahad",
  "avahahae",
  "avahahaf",
  "avahahag",
  "avahahah",
  "avahahai",
  "avahahaj",
  "avahaiaa",
  "avahaiab",
  "avahaiac",
  "avahaiad",
  "avahaiae",
  "avahaiaf",
  "avahaiag",
  "avahaiah",
  "avahaiai",
  "avahaiaj",
  "avahajaa",
  "avahajab",
  "avahajac",
  "avahajad",
  "avahajae",
  "avahajaf",
  "avahajag",
  "avahajah",
  "avahajai",
  "avahajaj",
  "avaiaaaa",
  "avaiaaab",
  "avaiaaac",
  "avaiaaad",
  "avaiaaae",
  "avaiaaaf",
  "avaiaaag",
  "avaiaaah",
  "avaiaaai",
  "avaiaaaj",
  "avaiabaa",
  "avaiabab",
  "avaiabac",
  "avaiabad",
  "avaiabae",
  "avaiabaf",
  "avaiabag",
  "avaiabah",
  "avaiabai",
  "avaiabaj",
  "avaiacaa",
  "avaiacab",
  "avaiacac",
  "avaiacad",
  "avaiacae",
  "avaiacaf",
  "avaiacag",
  "avaiacah",
  "avaiacai",
  "avaiacaj",
  "avaiadaa",
  "avaiadab",
  "avaiadac",
  "avaiadad",
  "avaiadae",
  "avaiadaf",
  "avaiadag",
  "avaiadah",
  "avaiadai",
  "avaiadaj",
  "avaiaeaa",
  "avaiaeab",
  "avaiaeac",
  "avaiaead",
  "avaiaeae",
  "avaiaeaf",
  "avaiaeag",
  "avaiaeah",
  "avaiaeai",
  "avaiaeaj",
  "avaiafaa",
  "avaiafab",
  "avaiafac",
  "avaiafad",
  "avaiafae",
  "avaiafaf",
  "avaiafag",
  "avaiafah",
  "avaiafai",
  "avaiafaj",
  "avaiagaa",
  "avaiagab",
  "avaiagac",
  "avaiagad",
  "avaiagae",
  "avaiagaf",
  "avaiagag",
  "avaiagah",
  "avaiagai",
  "avaiagaj",
  "avaiahaa",
  "avaiahab",
  "avaiahac",
  "avaiahad",
  "avaiahae",
  "avaiahaf",
  "avaiahag",
  "avaiahah",
  "avaiahai",
  "avaiahaj",
  "avaiaiaa",
  "avaiaiab",
  "avaiaiac",
  "avaiaiad",
  "avaiaiae",
  "avaiaiaf",
  "avaiaiag",
  "avaiaiah",
  "avaiaiai",
  "avaiaiaj",
  "avaiajaa",
  "avaiajab",
  "avaiajac",
  "avaiajad",
  "avaiajae",
  "avaiajaf",
  "avaiajag",
  "avaiajah",
  "avaiajai",
  "avaiajaj",
  "avajaaaa",
  "avajaaab",
  "avajaaac",
  "avajaaad",
  "avajaaae",
  "avajaaaf",
  "avajaaag",
  "avajaaah",
  "avajaaai",
  "avajaaaj",
  "avajabaa",
  "avajabab",
  "avajabac",
  "avajabad",
  "avajabae",
  "avajabaf",
  "avajabag",
  "avajabah",
  "avajabai",
  "avajabaj",
  "avajacaa",
  "avajacab",
  "avajacac",
  "avajacad",
  "avajacae",
  "avajacaf",
  "avajacag",
  "avajacah",
  "avajacai",
  "avajacaj",
  "avajadaa",
  "avajadab",
  "avajadac",
  "avajadad",
  "avajadae",
  "avajadaf",
  "avajadag",
  "avajadah",
  "avajadai",
  "avajadaj",
  "avajaeaa",
  "avajaeab",
  "avajaeac",
  "avajaead",
  "avajaeae",
  "avajaeaf",
  "avajaeag",
  "avajaeah",
  "avajaeai",
  "avajaeaj",
  "avajafaa",
  "avajafab",
  "avajafac",
  "avajafad",
  "avajafae",
  "avajafaf",
  "avajafag",
  "avajafah",
  "avajafai",
  "avajafaj",
  "avajagaa",
  "avajagab",
  "avajagac",
  "avajagad",
  "avajagae",
  "avajagaf",
  "avajagag",
  "avajagah",
  "avajagai",
  "avajagaj",
  "avajahaa",
  "avajahab",
  "avajahac",
  "avajahad",
  "avajahae",
  "avajahaf",
  "avajahag",
  "avajahah",
  "avajahai",
  "avajahaj",
  "avajaiaa",
  "avajaiab",
  "avajaiac",
  "avajaiad",
  "avajaiae",
  "avajaiaf",
  "avajaiag",
  "avajaiah",
  "avajaiai",
  "avajaiaj",
  "avajajaa",
  "avajajab",
  "avajajac",
  "avajajad",
  "avajajae",
  "avajajaf",
  "avajajag",
  "avajajah",
  "avajajai",
  "avajajaj",
  "awaaaaaa",
  "awaaaaab",
  "awaaaaac",
  "awaaaaad",
  "awaaaaae",
  "awaaaaaf",
  "awaaaaag",
  "awaaaaah",
  "awaaaaai",
  "awaaaaaj",
  "awaaabaa",
  "awaaabab",
  "awaaabac",
  "awaaabad",
  "awaaabae",
  "awaaabaf",
  "awaaabag",
  "awaaabah",
  "awaaabai",
  "awaaabaj",
  "awaaacaa",
  "awaaacab",
  "awaaacac",
  "awaaacad",
  "awaaacae",
  "awaaacaf",
  "awaaacag",
  "awaaacah",
  "awaaacai",
  "awaaacaj",
  "awaaadaa",
  "awaaadab",
  "awaaadac",
  "awaaadad",
  "awaaadae",
  "awaaadaf",
  "awaaadag",
  "awaaadah",
  "awaaadai",
  "awaaadaj",
  "awaaaeaa",
  "awaaaeab",
  "awaaaeac",
  "awaaaead",
  "awaaaeae",
  "awaaaeaf",
  "awaaaeag",
  "awaaaeah",
  "awaaaeai",
  "awaaaeaj",
  "awaaafaa",
  "awaaafab",
  "awaaafac",
  "awaaafad",
  "awaaafae",
  "awaaafaf",
  "awaaafag",
  "awaaafah",
  "awaaafai",
  "awaaafaj",
  "awaaagaa",
  "awaaagab",
  "awaaagac",
  "awaaagad",
  "awaaagae",
  "awaaagaf",
  "awaaagag",
  "awaaagah",
  "awaaagai",
  "awaaagaj",
  "awaaahaa",
  "awaaahab",
  "awaaahac",
  "awaaahad",
  "awaaahae",
  "awaaahaf",
  "awaaahag",
  "awaaahah",
  "awaaahai",
  "awaaahaj",
  "awaaaiaa",
  "awaaaiab",
  "awaaaiac",
  "awaaaiad",
  "awaaaiae",
  "awaaaiaf",
  "awaaaiag",
  "awaaaiah",
  "awaaaiai",
  "awaaaiaj",
  "awaaajaa",
  "awaaajab",
  "awaaajac",
  "awaaajad",
  "awaaajae",
  "awaaajaf",
  "awaaajag",
  "awaaajah",
  "awaaajai",
  "awaaajaj",
  "awabaaaa",
  "awabaaab",
  "awabaaac",
  "awabaaad",
  "awabaaae",
  "awabaaaf",
  "awabaaag",
  "awabaaah",
  "awabaaai",
  "awabaaaj",
  "awababaa",
  "awababab",
  "awababac",
  "awababad",
  "awababae",
  "awababaf",
  "awababag",
  "awababah",
  "awababai",
  "awababaj",
  "awabacaa",
  "awabacab",
  "awabacac",
  "awabacad",
  "awabacae",
  "awabacaf",
  "awabacag",
  "awabacah",
  "awabacai",
  "awabacaj",
  "awabadaa",
  "awabadab",
  "awabadac",
  "awabadad",
  "awabadae",
  "awabadaf",
  "awabadag",
  "awabadah",
  "awabadai",
  "awabadaj",
  "awabaeaa",
  "awabaeab",
  "awabaeac",
  "awabaead",
  "awabaeae",
  "awabaeaf",
  "awabaeag",
  "awabaeah",
  "awabaeai",
  "awabaeaj",
  "awabafaa",
  "awabafab",
  "awabafac",
  "awabafad",
  "awabafae",
  "awabafaf",
  "awabafag",
  "awabafah",
  "awabafai",
  "awabafaj",
  "awabagaa",
  "awabagab",
  "awabagac",
  "awabagad",
  "awabagae",
  "awabagaf",
  "awabagag",
  "awabagah",
  "awabagai",
  "awabagaj",
  "awabahaa",
  "awabahab",
  "awabahac",
  "awabahad",
  "awabahae",
  "awabahaf",
  "awabahag",
  "awabahah",
  "awabahai",
  "awabahaj",
  "awabaiaa",
  "awabaiab",
  "awabaiac",
  "awabaiad",
  "awabaiae",
  "awabaiaf",
  "awabaiag",
  "awabaiah",
  "awabaiai",
  "awabaiaj",
  "awabajaa",
  "awabajab",
  "awabajac",
  "awabajad",
  "awabajae",
  "awabajaf",
  "awabajag",
  "awabajah",
  "awabajai",
  "awabajaj",
  "awacaaaa",
  "awacaaab",
  "awacaaac",
  "awacaaad",
  "awacaaae",
  "awacaaaf",
  "awacaaag",
  "awacaaah",
  "awacaaai",
  "awacaaaj",
  "awacabaa",
  "awacabab",
  "awacabac",
  "awacabad",
  "awacabae",
  "awacabaf",
  "awacabag",
  "awacabah",
  "awacabai",
  "awacabaj",
  "awacacaa",
  "awacacab",
  "awacacac",
  "awacacad",
  "awacacae",
  "awacacaf",
  "awacacag",
  "awacacah",
  "awacacai",
  "awacacaj",
  "awacadaa",
  "awacadab",
  "awacadac",
  "awacadad",
  "awacadae",
  "awacadaf",
  "awacadag",
  "awacadah",
  "awacadai",
  "awacadaj",
  "awacaeaa",
  "awacaeab",
  "awacaeac",
  "awacaead",
  "awacaeae",
  "awacaeaf",
  "awacaeag",
  "awacaeah",
  "awacaeai",
  "awacaeaj",
  "awacafaa",
  "awacafab",
  "awacafac",
  "awacafad",
  "awacafae",
  "awacafaf",
  "awacafag",
  "awacafah",
  "awacafai",
  "awacafaj",
  "awacagaa",
  "awacagab",
  "awacagac",
  "awacagad",
  "awacagae",
  "awacagaf",
  "awacagag",
  "awacagah",
  "awacagai",
  "awacagaj",
  "awacahaa",
  "awacahab",
  "awacahac",
  "awacahad",
  "awacahae",
  "awacahaf",
  "awacahag",
  "awacahah",
  "awacahai",
  "awacahaj",
  "awacaiaa",
  "awacaiab",
  "awacaiac",
  "awacaiad",
  "awacaiae",
  "awacaiaf",
  "awacaiag",
  "awacaiah",
  "awacaiai",
  "awacaiaj",
  "awacajaa",
  "awacajab",
  "awacajac",
  "awacajad",
  "awacajae",
  "awacajaf",
  "awacajag",
  "awacajah",
  "awacajai",
  "awacajaj",
  "awadaaaa",
  "awadaaab",
  "awadaaac",
  "awadaaad",
  "awadaaae",
  "awadaaaf",
  "awadaaag",
  "awadaaah",
  "awadaaai",
  "awadaaaj",
  "awadabaa",
  "awadabab",
  "awadabac",
  "awadabad",
  "awadabae",
  "awadabaf",
  "awadabag",
  "awadabah",
  "awadabai",
  "awadabaj",
  "awadacaa",
  "awadacab",
  "awadacac",
  "awadacad",
  "awadacae",
  "awadacaf",
  "awadacag",
  "awadacah",
  "awadacai",
  "awadacaj",
  "awadadaa",
  "awadadab",
  "awadadac",
  "awadadad",
  "awadadae",
  "awadadaf",
  "awadadag",
  "awadadah",
  "awadadai",
  "awadadaj",
  "awadaeaa",
  "awadaeab",
  "awadaeac",
  "awadaead",
  "awadaeae",
  "awadaeaf",
  "awadaeag",
  "awadaeah",
  "awadaeai",
  "awadaeaj",
  "awadafaa",
  "awadafab",
  "awadafac",
  "awadafad",
  "awadafae",
  "awadafaf",
  "awadafag",
  "awadafah",
  "awadafai",
  "awadafaj",
  "awadagaa",
  "awadagab",
  "awadagac",
  "awadagad",
  "awadagae",
  "awadagaf",
  "awadagag",
  "awadagah",
  "awadagai",
  "awadagaj",
  "awadahaa",
  "awadahab",
  "awadahac",
  "awadahad",
  "awadahae",
  "awadahaf",
  "awadahag",
  "awadahah",
  "awadahai",
  "awadahaj",
  "awadaiaa",
  "awadaiab",
  "awadaiac",
  "awadaiad",
  "awadaiae",
  "awadaiaf",
  "awadaiag",
  "awadaiah",
  "awadaiai",
  "awadaiaj",
  "awadajaa",
  "awadajab",
  "awadajac",
  "awadajad",
  "awadajae",
  "awadajaf",
  "awadajag",
  "awadajah",
  "awadajai",
  "awadajaj",
  "awaeaaaa",
  "awaeaaab",
  "awaeaaac",
  "awaeaaad",
  "awaeaaae",
  "awaeaaaf",
  "awaeaaag",
  "awaeaaah",
  "awaeaaai",
  "awaeaaaj",
  "awaeabaa",
  "awaeabab",
  "awaeabac",
  "awaeabad",
  "awaeabae",
  "awaeabaf",
  "awaeabag",
  "awaeabah",
  "awaeabai",
  "awaeabaj",
  "awaeacaa",
  "awaeacab",
  "awaeacac",
  "awaeacad",
  "awaeacae",
  "awaeacaf",
  "awaeacag",
  "awaeacah",
  "awaeacai",
  "awaeacaj",
  "awaeadaa",
  "awaeadab",
  "awaeadac",
  "awaeadad",
  "awaeadae",
  "awaeadaf",
  "awaeadag",
  "awaeadah",
  "awaeadai",
  "awaeadaj",
  "awaeaeaa",
  "awaeaeab",
  "awaeaeac",
  "awaeaead",
  "awaeaeae",
  "awaeaeaf",
  "awaeaeag",
  "awaeaeah",
  "awaeaeai",
  "awaeaeaj",
  "awaeafaa",
  "awaeafab",
  "awaeafac",
  "awaeafad",
  "awaeafae",
  "awaeafaf",
  "awaeafag",
  "awaeafah",
  "awaeafai",
  "awaeafaj",
  "awaeagaa",
  "awaeagab",
  "awaeagac",
  "awaeagad",
  "awaeagae",
  "awaeagaf",
  "awaeagag",
  "awaeagah",
  "awaeagai",
  "awaeagaj",
  "awaeahaa",
  "awaeahab",
  "awaeahac",
  "awaeahad",
  "awaeahae",
  "awaeahaf",
  "awaeahag",
  "awaeahah",
  "awaeahai",
  "awaeahaj",
  "awaeaiaa",
  "awaeaiab",
  "awaeaiac",
  "awaeaiad",
  "awaeaiae",
  "awaeaiaf",
  "awaeaiag",
  "awaeaiah",
  "awaeaiai",
  "awaeaiaj",
  "awaeajaa",
  "awaeajab",
  "awaeajac",
  "awaeajad",
  "awaeajae",
  "awaeajaf",
  "awaeajag",
  "awaeajah",
  "awaeajai",
  "awaeajaj",
  "awafaaaa",
  "awafaaab",
  "awafaaac",
  "awafaaad",
  "awafaaae",
  "awafaaaf",
  "awafaaag",
  "awafaaah",
  "awafaaai",
  "awafaaaj",
  "awafabaa",
  "awafabab",
  "awafabac",
  "awafabad",
  "awafabae",
  "awafabaf",
  "awafabag",
  "awafabah",
  "awafabai",
  "awafabaj",
  "awafacaa",
  "awafacab",
  "awafacac",
  "awafacad",
  "awafacae",
  "awafacaf",
  "awafacag",
  "awafacah",
  "awafacai",
  "awafacaj",
  "awafadaa",
  "awafadab",
  "awafadac",
  "awafadad",
  "awafadae",
  "awafadaf",
  "awafadag",
  "awafadah",
  "awafadai",
  "awafadaj",
  "awafaeaa",
  "awafaeab",
  "awafaeac",
  "awafaead",
  "awafaeae",
  "awafaeaf",
  "awafaeag",
  "awafaeah",
  "awafaeai",
  "awafaeaj",
  "awafafaa",
  "awafafab",
  "awafafac",
  "awafafad",
  "awafafae",
  "awafafaf",
  "awafafag",
  "awafafah",
  "awafafai",
  "awafafaj",
  "awafagaa",
  "awafagab",
  "awafagac",
  "awafagad",
  "awafagae",
  "awafagaf",
  "awafagag",
  "awafagah",
  "awafagai",
  "awafagaj",
  "awafahaa",
  "awafahab",
  "awafahac",
  "awafahad",
  "awafahae",
  "awafahaf",
  "awafahag",
  "awafahah",
  "awafahai",
  "awafahaj",
  "awafaiaa",
  "awafaiab",
  "awafaiac",
  "awafaiad",
  "awafaiae",
  "awafaiaf",
  "awafaiag",
  "awafaiah",
  "awafaiai",
  "awafaiaj",
  "awafajaa",
  "awafajab",
  "awafajac",
  "awafajad",
  "awafajae",
  "awafajaf",
  "awafajag",
  "awafajah",
  "awafajai",
  "awafajaj",
  "awagaaaa",
  "awagaaab",
  "awagaaac",
  "awagaaad",
  "awagaaae",
  "awagaaaf",
  "awagaaag",
  "awagaaah",
  "awagaaai",
  "awagaaaj",
  "awagabaa",
  "awagabab",
  "awagabac",
  "awagabad",
  "awagabae",
  "awagabaf",
  "awagabag",
  "awagabah",
  "awagabai",
  "awagabaj",
  "awagacaa",
  "awagacab",
  "awagacac",
  "awagacad",
  "awagacae",
  "awagacaf",
  "awagacag",
  "awagacah",
  "awagacai",
  "awagacaj",
  "awagadaa",
  "awagadab",
  "awagadac",
  "awagadad",
  "awagadae",
  "awagadaf",
  "awagadag",
  "awagadah",
  "awagadai",
  "awagadaj",
  "awagaeaa",
  "awagaeab",
  "awagaeac",
  "awagaead",
  "awagaeae",
  "awagaeaf",
  "awagaeag",
  "awagaeah",
  "awagaeai",
  "awagaeaj",
  "awagafaa",
  "awagafab",
  "awagafac",
  "awagafad",
  "awagafae",
  "awagafaf",
  "awagafag",
  "awagafah",
  "awagafai",
  "awagafaj",
  "awagagaa",
  "awagagab",
  "awagagac",
  "awagagad",
  "awagagae",
  "awagagaf",
  "awagagag",
  "awagagah",
  "awagagai",
  "awagagaj",
  "awagahaa",
  "awagahab",
  "awagahac",
  "awagahad",
  "awagahae",
  "awagahaf",
  "awagahag",
  "awagahah",
  "awagahai",
  "awagahaj",
  "awagaiaa",
  "awagaiab",
  "awagaiac",
  "awagaiad",
  "awagaiae",
  "awagaiaf",
  "awagaiag",
  "awagaiah",
  "awagaiai",
  "awagaiaj",
  "awagajaa",
  "awagajab",
  "awagajac",
  "awagajad",
  "awagajae",
  "awagajaf",
  "awagajag",
  "awagajah",
  "awagajai",
  "awagajaj",
  "awahaaaa",
  "awahaaab",
  "awahaaac",
  "awahaaad",
  "awahaaae",
  "awahaaaf",
  "awahaaag",
  "awahaaah",
  "awahaaai",
  "awahaaaj",
  "awahabaa",
  "awahabab",
  "awahabac",
  "awahabad",
  "awahabae",
  "awahabaf",
  "awahabag",
  "awahabah",
  "awahabai",
  "awahabaj",
  "awahacaa",
  "awahacab",
  "awahacac",
  "awahacad",
  "awahacae",
  "awahacaf",
  "awahacag",
  "awahacah",
  "awahacai",
  "awahacaj",
  "awahadaa",
  "awahadab",
  "awahadac",
  "awahadad",
  "awahadae",
  "awahadaf",
  "awahadag",
  "awahadah",
  "awahadai",
  "awahadaj",
  "awahaeaa",
  "awahaeab",
  "awahaeac",
  "awahaead",
  "awahaeae",
  "awahaeaf",
  "awahaeag",
  "awahaeah",
  "awahaeai",
  "awahaeaj",
  "awahafaa",
  "awahafab",
  "awahafac",
  "awahafad",
  "awahafae",
  "awahafaf",
  "awahafag",
  "awahafah",
  "awahafai",
  "awahafaj",
  "awahagaa",
  "awahagab",
  "awahagac",
  "awahagad",
  "awahagae",
  "awahagaf",
  "awahagag",
  "awahagah",
  "awahagai",
  "awahagaj",
  "awahahaa",
  "awahahab",
  "awahahac",
  "awahahad",
  "awahahae",
  "awahahaf",
  "awahahag",
  "awahahah",
  "awahahai",
  "awahahaj",
  "awahaiaa",
  "awahaiab",
  "awahaiac",
  "awahaiad",
  "awahaiae",
  "awahaiaf",
  "awahaiag",
  "awahaiah",
  "awahaiai",
  "awahaiaj",
  "awahajaa",
  "awahajab",
  "awahajac",
  "awahajad",
  "awahajae",
  "awahajaf",
  "awahajag",
  "awahajah",
  "awahajai",
  "awahajaj",
  "awaiaaaa",
  "awaiaaab",
  "awaiaaac",
  "awaiaaad",
  "awaiaaae",
  "awaiaaaf",
  "awaiaaag",
  "awaiaaah",
  "awaiaaai",
  "awaiaaaj",
  "awaiabaa",
  "awaiabab",
  "awaiabac",
  "awaiabad",
  "awaiabae",
  "awaiabaf",
  "awaiabag",
  "awaiabah",
  "awaiabai",
  "awaiabaj",
  "awaiacaa",
  "awaiacab",
  "awaiacac",
  "awaiacad",
  "awaiacae",
  "awaiacaf",
  "awaiacag",
  "awaiacah",
  "awaiacai",
  "awaiacaj",
  "awaiadaa",
  "awaiadab",
  "awaiadac",
  "awaiadad",
  "awaiadae",
  "awaiadaf",
  "awaiadag",
  "awaiadah",
  "awaiadai",
  "awaiadaj",
  "awaiaeaa",
  "awaiaeab",
  "awaiaeac",
  "awaiaead",
  "awaiaeae",
  "awaiaeaf",
  "awaiaeag",
  "awaiaeah",
  "awaiaeai",
  "awaiaeaj",
  "awaiafaa",
  "awaiafab",
  "awaiafac",
  "awaiafad",
  "awaiafae",
  "awaiafaf",
  "awaiafag",
  "awaiafah",
  "awaiafai",
  "awaiafaj",
  "awaiagaa",
  "awaiagab",
  "awaiagac",
  "awaiagad",
  "awaiagae",
  "awaiagaf",
  "awaiagag",
  "awaiagah",
  "awaiagai",
  "awaiagaj",
  "awaiahaa",
  "awaiahab",
  "awaiahac",
  "awaiahad",
  "awaiahae",
  "awaiahaf",
  "awaiahag",
  "awaiahah",
  "awaiahai",
  "awaiahaj",
  "awaiaiaa",
  "awaiaiab",
  "awaiaiac",
  "awaiaiad",
  "awaiaiae",
  "awaiaiaf",
  "awaiaiag",
  "awaiaiah",
  "awaiaiai",
  "awaiaiaj",
  "awaiajaa",
  "awaiajab",
  "awaiajac",
  "awaiajad",
  "awaiajae",
  "awaiajaf",
  "awaiajag",
  "awaiajah",
  "awaiajai",
  "awaiajaj",
  "awajaaaa",
  "awajaaab",
  "awajaaac",
  "awajaaad",
  "awajaaae",
  "awajaaaf",
  "awajaaag",
  "awajaaah",
  "awajaaai",
  "awajaaaj",
  "awajabaa",
  "awajabab",
  "awajabac",
  "awajabad",
  "awajabae",
  "awajabaf",
  "awajabag",
  "awajabah",
  "awajabai",
  "awajabaj",
  "awajacaa",
  "awajacab",
  "awajacac",
  "awajacad",
  "awajacae",
  "awajacaf",
  "awajacag",
  "awajacah",
  "awajacai",
  "awajacaj",
  "awajadaa",
  "awajadab",
  "awajadac",
  "awajadad",
  "awajadae",
  "awajadaf",
  "awajadag",
  "awajadah",
  "awajadai",
  "awajadaj",
  "awajaeaa",
  "awajaeab",
  "awajaeac",
  "awajaead",
  "awajaeae",
  "awajaeaf",
  "awajaeag",
  "awajaeah",
  "awajaeai",
  "awajaeaj",
  "awajafaa",
  "awajafab",
  "awajafac",
  "awajafad",
  "awajafae",
  "awajafaf",
  "awajafag",
  "awajafah",
  "awajafai",
  "awajafaj",
  "awajagaa",
  "awajagab",
  "awajagac",
  "awajagad",
  "awajagae",
  "awajagaf",
  "awajagag",
  "awajagah",
  "awajagai",
  "awajagaj",
  "awajahaa",
  "awajahab",
  "awajahac",
  "awajahad",
  "awajahae",
  "awajahaf",
  "awajahag",
  "awajahah",
  "awajahai",
  "awajahaj",
  "awajaiaa",
  "awajaiab",
  "awajaiac",
  "awajaiad",
  "awajaiae",
  "awajaiaf",
  "awajaiag",
  "awajaiah",
  "awajaiai",
  "awajaiaj",
  "awajajaa",
  "awajajab",
  "awajajac",
  "awajajad",
  "awajajae",
  "awajajaf",
  "awajajag",
  "awajajah",
  "awajajai",
  "awajajaj",
  "axaaaaaa",
  "axaaaaab",
  "axaaaaac",
  "axaaaaad",
  "axaaaaae",
  "axaaaaaf",
  "axaaaaag",
  "axaaaaah",
  "axaaaaai",
  "axaaaaaj",
  "axaaabaa",
  "axaaabab",
  "axaaabac",
  "axaaabad",
  "axaaabae",
  "axaaabaf",
  "axaaabag",
  "axaaabah",
  "axaaabai",
  "axaaabaj",
  "axaaacaa",
  "axaaacab",
  "axaaacac",
  "axaaacad",
  "axaaacae",
  "axaaacaf",
  "axaaacag",
  "axaaacah",
  "axaaacai",
  "axaaacaj",
  "axaaadaa",
  "axaaadab",
  "axaaadac",
  "axaaadad",
  "axaaadae",
  "axaaadaf",
  "axaaadag",
  "axaaadah",
  "axaaadai",
  "axaaadaj",
  "axaaaeaa",
  "axaaaeab",
  "axaaaeac",
  "axaaaead",
  "axaaaeae",
  "axaaaeaf",
  "axaaaeag",
  "axaaaeah",
  "axaaaeai",
  "axaaaeaj",
  "axaaafaa",
  "axaaafab",
  "axaaafac",
  "axaaafad",
  "axaaafae",
  "axaaafaf",
  "axaaafag",
  "axaaafah",
  "axaaafai",
  "axaaafaj",
  "axaaagaa",
  "axaaagab",
  "axaaagac",
  "axaaagad",
  "axaaagae",
  "axaaagaf",
  "axaaagag",
  "axaaagah",
  "axaaagai",
  "axaaagaj",
  "axaaahaa",
  "axaaahab",
  "axaaahac",
  "axaaahad",
  "axaaahae",
  "axaaahaf",
  "axaaahag",
  "axaaahah",
  "axaaahai",
  "axaaahaj",
  "axaaaiaa",
  "axaaaiab",
  "axaaaiac",
  "axaaaiad",
  "axaaaiae",
  "axaaaiaf",
  "axaaaiag",
  "axaaaiah",
  "axaaaiai",
  "axaaaiaj",
  "axaaajaa",
  "axaaajab",
  "axaaajac",
  "axaaajad",
  "axaaajae",
  "axaaajaf",
  "axaaajag",
  "axaaajah",
  "axaaajai",
  "axaaajaj",
  "axabaaaa",
  "axabaaab",
  "axabaaac",
  "axabaaad",
  "axabaaae",
  "axabaaaf",
  "axabaaag",
  "axabaaah",
  "axabaaai",
  "axabaaaj",
  "axababaa",
  "axababab",
  "axababac",
  "axababad",
  "axababae",
  "axababaf",
  "axababag",
  "axababah",
  "axababai",
  "axababaj",
  "axabacaa",
  "axabacab",
  "axabacac",
  "axabacad",
  "axabacae",
  "axabacaf",
  "axabacag",
  "axabacah",
  "axabacai",
  "axabacaj",
  "axabadaa",
  "axabadab",
  "axabadac",
  "axabadad",
  "axabadae",
  "axabadaf",
  "axabadag",
  "axabadah",
  "axabadai",
  "axabadaj",
  "axabaeaa",
  "axabaeab",
  "axabaeac",
  "axabaead",
  "axabaeae",
  "axabaeaf",
  "axabaeag",
  "axabaeah",
  "axabaeai",
  "axabaeaj",
  "axabafaa",
  "axabafab",
  "axabafac",
  "axabafad",
  "axabafae",
  "axabafaf",
  "axabafag",
  "axabafah",
  "axabafai",
  "axabafaj",
  "axabagaa",
  "axabagab",
  "axabagac",
  "axabagad",
  "axabagae",
  "axabagaf",
  "axabagag",
  "axabagah",
  "axabagai",
  "axabagaj",
  "axabahaa",
  "axabahab",
  "axabahac",
  "axabahad",
  "axabahae",
  "axabahaf",
  "axabahag",
  "axabahah",
  "axabahai",
  "axabahaj",
  "axabaiaa",
  "axabaiab",
  "axabaiac",
  "axabaiad",
  "axabaiae",
  "axabaiaf",
  "axabaiag",
  "axabaiah",
  "axabaiai",
  "axabaiaj",
  "axabajaa",
  "axabajab",
  "axabajac",
  "axabajad",
  "axabajae",
  "axabajaf",
  "axabajag",
  "axabajah",
  "axabajai",
  "axabajaj",
  "axacaaaa",
  "axacaaab",
  "axacaaac",
  "axacaaad",
  "axacaaae",
  "axacaaaf",
  "axacaaag",
  "axacaaah",
  "axacaaai",
  "axacaaaj",
  "axacabaa",
  "axacabab",
  "axacabac",
  "axacabad",
  "axacabae",
  "axacabaf",
  "axacabag",
  "axacabah",
  "axacabai",
  "axacabaj",
  "axacacaa",
  "axacacab",
  "axacacac",
  "axacacad",
  "axacacae",
  "axacacaf",
  "axacacag",
  "axacacah",
  "axacacai",
  "axacacaj",
  "axacadaa",
  "axacadab",
  "axacadac",
  "axacadad",
  "axacadae",
  "axacadaf",
  "axacadag",
  "axacadah",
  "axacadai",
  "axacadaj",
  "axacaeaa",
  "axacaeab",
  "axacaeac",
  "axacaead",
  "axacaeae",
  "axacaeaf",
  "axacaeag",
  "axacaeah",
  "axacaeai",
  "axacaeaj",
  "axacafaa",
  "axacafab",
  "axacafac",
  "axacafad",
  "axacafae",
  "axacafaf",
  "axacafag",
  "axacafah",
  "axacafai",
  "axacafaj",
  "axacagaa",
  "axacagab",
  "axacagac",
  "axacagad",
  "axacagae",
  "axacagaf",
  "axacagag",
  "axacagah",
  "axacagai",
  "axacagaj",
  "axacahaa",
  "axacahab",
  "axacahac",
  "axacahad",
  "axacahae",
  "axacahaf",
  "axacahag",
  "axacahah",
  "axacahai",
  "axacahaj",
  "axacaiaa",
  "axacaiab",
  "axacaiac",
  "axacaiad",
  "axacaiae",
  "axacaiaf",
  "axacaiag",
  "axacaiah",
  "axacaiai",
  "axacaiaj",
  "axacajaa",
  "axacajab",
  "axacajac",
  "axacajad",
  "axacajae",
  "axacajaf",
  "axacajag",
  "axacajah",
  "axacajai",
  "axacajaj",
  "axadaaaa",
  "axadaaab",
  "axadaaac",
  "axadaaad",
  "axadaaae",
  "axadaaaf",
  "axadaaag",
  "axadaaah",
  "axadaaai",
  "axadaaaj",
  "axadabaa",
  "axadabab",
  "axadabac",
  "axadabad",
  "axadabae",
  "axadabaf",
  "axadabag",
  "axadabah",
  "axadabai",
  "axadabaj",
  "axadacaa",
  "axadacab",
  "axadacac",
  "axadacad",
  "axadacae",
  "axadacaf",
  "axadacag",
  "axadacah",
  "axadacai",
  "axadacaj",
  "axadadaa",
  "axadadab",
  "axadadac",
  "axadadad",
  "axadadae",
  "axadadaf",
  "axadadag",
  "axadadah",
  "axadadai",
  "axadadaj",
  "axadaeaa",
  "axadaeab",
  "axadaeac",
  "axadaead",
  "axadaeae",
  "axadaeaf",
  "axadaeag",
  "axadaeah",
  "axadaeai",
  "axadaeaj",
  "axadafaa",
  "axadafab",
  "axadafac",
  "axadafad",
  "axadafae",
  "axadafaf",
  "axadafag",
  "axadafah",
  "axadafai",
  "axadafaj",
  "axadagaa",
  "axadagab",
  "axadagac",
  "axadagad",
  "axadagae",
  "axadagaf",
  "axadagag",
  "axadagah",
  "axadagai",
  "axadagaj",
  "axadahaa",
  "axadahab",
  "axadahac",
  "axadahad",
  "axadahae",
  "axadahaf",
  "axadahag",
  "axadahah",
  "axadahai",
  "axadahaj",
  "axadaiaa",
  "axadaiab",
  "axadaiac",
  "axadaiad",
  "axadaiae",
  "axadaiaf",
  "axadaiag",
  "axadaiah",
  "axadaiai",
  "axadaiaj",
  "axadajaa",
  "axadajab",
  "axadajac",
  "axadajad",
  "axadajae",
  "axadajaf",
  "axadajag",
  "axadajah",
  "axadajai",
  "axadajaj",
  "axaeaaaa",
  "axaeaaab",
  "axaeaaac",
  "axaeaaad",
  "axaeaaae",
  "axaeaaaf",
  "axaeaaag",
  "axaeaaah",
  "axaeaaai",
  "axaeaaaj",
  "axaeabaa",
  "axaeabab",
  "axaeabac",
  "axaeabad",
  "axaeabae",
  "axaeabaf",
  "axaeabag",
  "axaeabah",
  "axaeabai",
  "axaeabaj",
  "axaeacaa",
  "axaeacab",
  "axaeacac",
  "axaeacad",
  "axaeacae",
  "axaeacaf",
  "axaeacag",
  "axaeacah",
  "axaeacai",
  "axaeacaj",
  "axaeadaa",
  "axaeadab",
  "axaeadac",
  "axaeadad",
  "axaeadae",
  "axaeadaf",
  "axaeadag",
  "axaeadah",
  "axaeadai",
  "axaeadaj",
  "axaeaeaa",
  "axaeaeab",
  "axaeaeac",
  "axaeaead",
  "axaeaeae",
  "axaeaeaf",
  "axaeaeag",
  "axaeaeah",
  "axaeaeai",
  "axaeaeaj",
  "axaeafaa",
  "axaeafab",
  "axaeafac",
  "axaeafad",
  "axaeafae",
  "axaeafaf",
  "axaeafag",
  "axaeafah",
  "axaeafai",
  "axaeafaj",
  "axaeagaa",
  "axaeagab",
  "axaeagac",
  "axaeagad",
  "axaeagae",
  "axaeagaf",
  "axaeagag",
  "axaeagah",
  "axaeagai",
  "axaeagaj",
  "axaeahaa",
  "axaeahab",
  "axaeahac",
  "axaeahad",
  "axaeahae",
  "axaeahaf",
  "axaeahag",
  "axaeahah",
  "axaeahai",
  "axaeahaj",
  "axaeaiaa",
  "axaeaiab",
  "axaeaiac",
  "axaeaiad",
  "axaeaiae",
  "axaeaiaf",
  "axaeaiag",
  "axaeaiah",
  "axaeaiai",
  "axaeaiaj",
  "axaeajaa",
  "axaeajab",
  "axaeajac",
  "axaeajad",
  "axaeajae",
  "axaeajaf",
  "axaeajag",
  "axaeajah",
  "axaeajai",
  "axaeajaj",
  "axafaaaa",
  "axafaaab",
  "axafaaac",
  "axafaaad",
  "axafaaae",
  "axafaaaf",
  "axafaaag",
  "axafaaah",
  "axafaaai",
  "axafaaaj",
  "axafabaa",
  "axafabab",
  "axafabac",
  "axafabad",
  "axafabae",
  "axafabaf",
  "axafabag",
  "axafabah",
  "axafabai",
  "axafabaj",
  "axafacaa",
  "axafacab",
  "axafacac",
  "axafacad",
  "axafacae",
  "axafacaf",
  "axafacag",
  "axafacah",
  "axafacai",
  "axafacaj",
  "axafadaa",
  "axafadab",
  "axafadac",
  "axafadad",
  "axafadae",
  "axafadaf",
  "axafadag",
  "axafadah",
  "axafadai",
  "axafadaj",
  "axafaeaa",
  "axafaeab",
  "axafaeac",
  "axafaead",
  "axafaeae",
  "axafaeaf",
  "axafaeag",
  "axafaeah",
  "axafaeai",
  "axafaeaj",
  "axafafaa",
  "axafafab",
  "axafafac",
  "axafafad",
  "axafafae",
  "axafafaf",
  "axafafag",
  "axafafah",
  "axafafai",
  "axafafaj",
  "axafagaa",
  "axafagab",
  "axafagac",
  "axafagad",
  "axafagae",
  "axafagaf",
  "axafagag",
  "axafagah",
  "axafagai",
  "axafagaj",
  "axafahaa",
  "axafahab",
  "axafahac",
  "axafahad",
  "axafahae",
  "axafahaf",
  "axafahag",
  "axafahah",
  "axafahai",
  "axafahaj",
  "axafaiaa",
  "axafaiab",
  "axafaiac",
  "axafaiad",
  "axafaiae",
  "axafaiaf",
  "axafaiag",
  "axafaiah",
  "axafaiai",
  "axafaiaj",
  "axafajaa",
  "axafajab",
  "axafajac",
  "axafajad",
  "axafajae",
  "axafajaf",
  "axafajag",
  "axafajah",
  "axafajai",
  "axafajaj",
  "axagaaaa",
  "axagaaab",
  "axagaaac",
  "axagaaad",
  "axagaaae",
  "axagaaaf",
  "axagaaag",
  "axagaaah",
  "axagaaai",
  "axagaaaj",
  "axagabaa",
  "axagabab",
  "axagabac",
  "axagabad",
  "axagabae",
  "axagabaf",
  "axagabag",
  "axagabah",
  "axagabai",
  "axagabaj",
  "axagacaa",
  "axagacab",
  "axagacac",
  "axagacad",
  "axagacae",
  "axagacaf",
  "axagacag",
  "axagacah",
  "axagacai",
  "axagacaj",
  "axagadaa",
  "axagadab",
  "axagadac",
  "axagadad",
  "axagadae",
  "axagadaf",
  "axagadag",
  "axagadah",
  "axagadai",
  "axagadaj",
  "axagaeaa",
  "axagaeab",
  "axagaeac",
  "axagaead",
  "axagaeae",
  "axagaeaf",
  "axagaeag",
  "axagaeah",
  "axagaeai",
  "axagaeaj",
  "axagafaa",
  "axagafab",
  "axagafac",
  "axagafad",
  "axagafae",
  "axagafaf",
  "axagafag",
  "axagafah",
  "axagafai",
  "axagafaj",
  "axagagaa",
  "axagagab",
  "axagagac",
  "axagagad",
  "axagagae",
  "axagagaf",
  "axagagag",
  "axagagah",
  "axagagai",
  "axagagaj",
  "axagahaa",
  "axagahab",
  "axagahac",
  "axagahad",
  "axagahae",
  "axagahaf",
  "axagahag",
  "axagahah",
  "axagahai",
  "axagahaj",
  "axagaiaa",
  "axagaiab",
  "axagaiac",
  "axagaiad",
  "axagaiae",
  "axagaiaf",
  "axagaiag",
  "axagaiah",
  "axagaiai",
  "axagaiaj",
  "axagajaa",
  "axagajab",
  "axagajac",
  "axagajad",
  "axagajae",
  "axagajaf",
  "axagajag",
  "axagajah",
  "axagajai",
  "axagajaj",
  "axahaaaa",
  "axahaaab",
  "axahaaac",
  "axahaaad",
  "axahaaae",
  "axahaaaf",
  "axahaaag",
  "axahaaah",
  "axahaaai",
  "axahaaaj",
  "axahabaa",
  "axahabab",
  "axahabac",
  "axahabad",
  "axahabae",
  "axahabaf",
  "axahabag",
  "axahabah",
  "axahabai",
  "axahabaj",
  "axahacaa",
  "axahacab",
  "axahacac",
  "axahacad",
  "axahacae",
  "axahacaf",
  "axahacag",
  "axahacah",
  "axahacai",
  "axahacaj",
  "axahadaa",
  "axahadab",
  "axahadac",
  "axahadad",
  "axahadae",
  "axahadaf",
  "axahadag",
  "axahadah",
  "axahadai",
  "axahadaj",
  "axahaeaa",
  "axahaeab",
  "axahaeac",
  "axahaead",
  "axahaeae",
  "axahaeaf",
  "axahaeag",
  "axahaeah",
  "axahaeai",
  "axahaeaj",
  "axahafaa",
  "axahafab",
  "axahafac",
  "axahafad",
  "axahafae",
  "axahafaf",
  "axahafag",
  "axahafah",
  "axahafai",
  "axahafaj",
  "axahagaa",
  "axahagab",
  "axahagac",
  "axahagad",
  "axahagae",
  "axahagaf",
  "axahagag",
  "axahagah",
  "axahagai",
  "axahagaj",
  "axahahaa",
  "axahahab",
  "axahahac",
  "axahahad",
  "axahahae",
  "axahahaf",
  "axahahag",
  "axahahah",
  "axahahai",
  "axahahaj",
  "axahaiaa",
  "axahaiab",
  "axahaiac",
  "axahaiad",
  "axahaiae",
  "axahaiaf",
  "axahaiag",
  "axahaiah",
  "axahaiai",
  "axahaiaj",
  "axahajaa",
  "axahajab",
  "axahajac",
  "axahajad",
  "axahajae",
  "axahajaf",
  "axahajag",
  "axahajah",
  "axahajai",
  "axahajaj",
  "axaiaaaa",
  "axaiaaab",
  "axaiaaac",
  "axaiaaad",
  "axaiaaae",
  "axaiaaaf",
  "axaiaaag",
  "axaiaaah",
  "axaiaaai",
  "axaiaaaj",
  "axaiabaa",
  "axaiabab",
  "axaiabac",
  "axaiabad",
  "axaiabae",
  "axaiabaf",
  "axaiabag",
  "axaiabah",
  "axaiabai",
  "axaiabaj",
  "axaiacaa",
  "axaiacab",
  "axaiacac",
  "axaiacad",
  "axaiacae",
  "axaiacaf",
  "axaiacag",
  "axaiacah",
  "axaiacai",
  "axaiacaj",
  "axaiadaa",
  "axaiadab",
  "axaiadac",
  "axaiadad",
  "axaiadae",
  "axaiadaf",
  "axaiadag",
  "axaiadah",
  "axaiadai",
  "axaiadaj",
  "axaiaeaa",
  "axaiaeab",
  "axaiaeac",
  "axaiaead",
  "axaiaeae",
  "axaiaeaf",
  "axaiaeag",
  "axaiaeah",
  "axaiaeai",
  "axaiaeaj",
  "axaiafaa",
  "axaiafab",
  "axaiafac",
  "axaiafad",
  "axaiafae",
  "axaiafaf",
  "axaiafag",
  "axaiafah",
  "axaiafai",
  "axaiafaj",
  "axaiagaa",
  "axaiagab",
  "axaiagac",
  "axaiagad",
  "axaiagae",
  "axaiagaf",
  "axaiagag",
  "axaiagah",
  "axaiagai",
  "axaiagaj",
  "axaiahaa",
  "axaiahab",
  "axaiahac",
  "axaiahad",
  "axaiahae",
  "axaiahaf",
  "axaiahag",
  "axaiahah",
  "axaiahai",
  "axaiahaj",
  "axaiaiaa",
  "axaiaiab",
  "axaiaiac",
  "axaiaiad",
  "axaiaiae",
  "axaiaiaf",
  "axaiaiag",
  "axaiaiah",
  "axaiaiai",
  "axaiaiaj",
  "axaiajaa",
  "axaiajab",
  "axaiajac",
  "axaiajad",
  "axaiajae",
  "axaiajaf",
  "axaiajag",
  "axaiajah",
  "axaiajai",
  "axaiajaj",
  "axajaaaa",
  "axajaaab",
  "axajaaac",
  "axajaaad",
  "axajaaae",
  "axajaaaf",
  "axajaaag",
  "axajaaah",
  "axajaaai",
  "axajaaaj",
  "axajabaa",
  "axajabab",
  "axajabac",
  "axajabad",
  "axajabae",
  "axajabaf",
  "axajabag",
  "axajabah",
  "axajabai",
  "axajabaj",
  "axajacaa",
  "axajacab",
  "axajacac",
  "axajacad",
  "axajacae",
  "axajacaf",
  "axajacag",
  "axajacah",
  "axajacai",
  "axajacaj",
  "axajadaa",
  "axajadab",
  "axajadac",
  "axajadad",
  "axajadae",
  "axajadaf",
  "axajadag",
  "axajadah",
  "axajadai",
  "axajadaj",
  "axajaeaa",
  "axajaeab",
  "axajaeac",
  "axajaead",
  "axajaeae",
  "axajaeaf",
  "axajaeag",
  "axajaeah",
  "axajaeai",
  "axajaeaj",
  "axajafaa",
  "axajafab",
  "axajafac",
  "axajafad",
  "axajafae",
  "axajafaf",
  "axajafag",
  "axajafah",
  "axajafai",
  "axajafaj",
  "axajagaa",
  "axajagab",
  "axajagac",
  "axajagad",
  "axajagae",
  "axajagaf",
  "axajagag",
  "axajagah",
  "axajagai",
  "axajagaj",
  "axajahaa",
  "axajahab",
  "axajahac",
  "axajahad",
  "axajahae",
  "axajahaf",
  "axajahag",
  "axajahah",
  "axajahai",
  "axajahaj",
  "axajaiaa",
  "axajaiab",
  "axajaiac",
  "axajaiad",
  "axajaiae",
  "axajaiaf",
  "axajaiag",
  "axajaiah",
  "axajaiai",
  "axajaiaj",
  "axajajaa",
  "axajajab",
  "axajajac",
  "axajajad",
  "axajajae",
  "axajajaf",
  "axajajag",
  "axajajah",
  "axajajai",
  "axajajaj",
  "bdahaiaa",
  "bdahaiab",
  "bdahaiac",
  "bdahaiad",
  "bdahaiae",
  "bdahaiaf",
  "bdahaiag",
  "bdahaiah",
  "bdahaiai",
  "bdahaiaj",
  "beabaiaa",
  "beabaiab",
  "beabaiac",
  "beabaiad",
  "beabaiae",
  "beabaiaf",
  "beabaiag",
  "beabaiah",
  "beabaiai",
  "beabaiaj",
  "beabajaa",
  "beabajab",
  "beabajac",
  "beabajad",
  "beabajae",
  "beabajaf",
  "beabajag",
  "beabajah",
  "beabajai",
  "beabajaj",
  "beacaaaa",
  "beacaaab",
  "beacaaac",
  "beacaaad",
  "beacaaae",
  "beacaaaf",
  "beacaaag",
  "beacaaah",
  "beacaaai",
  "beacaaaj",
  "beacabaa",
  "beacabab",
  "beacabac",
  "beacabad",
  "beacabae",
  "beacabaf",
  "beacabag",
  "beacabah",
  "beacabai",
  "beacabaj",
  "beacacaa",
  "beacacab",
  "beacacac",
  "beacacad",
  "beacacae",
  "beacacaf",
  "beacacag",
  "beacacah",
  "beacacai",
  "beacacaj",
  "beacadaa",
  "beacadab",
  "beacadac",
  "beacadad",
  "beacadae",
  "beacadaf",
  "beacadag",
  "beacadah",
  "beacadai",
  "beacadaj",
  "beacaeaa",
  "beacaeab",
  "beacaeac",
  "beacaead",
  "beacaeae",
  "beacaeaf",
  "beacaeag",
  "beacaeah",
  "beacaeai",
  "beacaeaj",
  "bjadadaa",
  "bjadadab",
  "bjadadac",
  "bjadadad",
  "bjadadae",
  "bjadadaf",
  "bjadadag",
  "bjadadah",
  "bjadadai",
  "bjadadaj",
  "bjadaeaa",
  "bjadaeab",
  "bjadaeac",
  "bjadaead",
  "bjadaeae",
  "bjadaeaf",
  "bjadaeag",
  "bjadaeah",
  "bjadaeai",
  "bjadaeaj",
  "bjaeabaa",
  "bjaeabab",
  "bjaeabac",
  "bjaeabad",
  "bjaeabae",
  "bjaeabaf",
  "bjaeabag",
  "bjaeabah",
  "bjaeabai",
  "bjaeabaj",
  "bjaeacaa",
  "bjaeacab",
  "bjaeacac",
  "bjaeacad",
  "bjaeacae",
  "bjaeacaf",
  "bjaeacag",
  "bjaeacah",
  "bjaeacai",
  "bjaeacaj",
  "bjaeadaa",
  "bjaeadab",
  "bjaeadac",
  "bjaeadad",
  "bjaeadae",
  "bjaeadaf",
  "bjaeadag",
  "bjaeadah",
  "bjaeadai",
  "bjaeadaj",
  "bjafacaa",
  "bjafacab",
  "bjafacac",
  "bjafacad",
  "bjafacae",
  "bjafacaf",
  "bjafacag",
  "bjafacah",
  "bjafacai",
  "bjafacaj",
  "blacaiaa",
  "blacaiab",
  "blacaiac",
  "blacaiad",
  "blacaiae",
  "blacaiaf",
  "blacaiag",
  "blacaiah",
  "blacaiai",
  "blacaiaj",
  "bmacabaa",
  "bmacabab",
  "bmacabac",
  "bmacabad",
  "bmacabae",
  "bmacabaf",
  "bmacabag",
  "bmacabah",
  "bmacabai",
  "bmacabaj",
  "boahabaa",
  "boahabab",
  "boahabac",
  "boahabad",
  "boahabae",
  "boahabaf",
  "boahabag",
  "boahabah",
  "boahabai",
  "boahabaj",
  "boahacaa",
  "boahacab",
  "boahacac",
  "boahacad",
  "boahacae",
  "boahacaf",
  "boahacag",
  "boahacah",
  "boahacai",
  "boahacaj",
  "boahadaa",
  "boahadab",
  "boahadac",
  "boahadad",
  "boahadae",
  "boahadaf",
  "boahadag",
  "boahadah",
  "boahadai",
  "boahadaj",
  "boahaeaa",
  "boahaeab",
  "boahaeac",
  "boahaead",
  "boahaeae",
  "boahaeaf",
  "boahaeag",
  "boahaeah",
  "boahaeai",
  "boahaeaj",
  "boahafaa",
  "boahafab",
  "boahafac",
  "boahafad",
  "boahafae",
  "boahafaf",
  "boahafag",
  "boahafah",
  "boahafai",
  "boahafaj",
  "boahagaa",
  "boahagab",
  "boahagac",
  "boahagad",
  "boahagae",
  "boahagaf",
  "boahagag",
  "boahagah",
  "boahagai",
  "boahagaj",
  "boahahaa",
  "boahahab",
  "boahahac",
  "boahahad",
  "boahahae",
  "boahahaf",
  "boahahag",
  "boahahah",
  "boahahai",
  "boahahaj",
  "boahaiaa",
  "boahaiab",
  "boahaiac",
  "boahaiad",
  "boahaiae",
  "boahaiaf",
  "boahaiag",
  "boahaiah",
  "boahaiai",
  "boahaiaj",
  "boahajaa",
  "boahajab",
  "boahajac",
  "boahajad",
  "boahajae",
  "boahajaf",
  "boahajag",
  "boahajah",
  "boahajai",
  "boahajaj",
  "boaiaaaa",
  "boaiaaab",
  "boaiaaac",
  "boaiaaad",
  "boaiaaae",
  "boaiaaaf",
  "boaiaaag",
  "boaiaaah",
  "boaiaaai",
  "boaiaaaj",
  "boaiabaa",
  "boaiabab",
  "boaiabac",
  "boaiabad",
  "boaiabae",
  "boaiabaf",
  "boaiabag",
  "boaiabah",
  "boaiabai",
  "boaiabaj",
  "boaiacaa",
  "boaiacab",
  "boaiacac",
  "boaiacad",
  "boaiacae",
  "boaiacaf",
  "boaiacag",
  "boaiacah",
  "boaiacai",
  "boaiacaj",
  "boaiadaa",
  "boaiadab",
  "boaiadac",
  "boaiadad",
  "boaiadae",
  "boaiadaf",
  "boaiadag",
  "boaiadah",
  "boaiadai",
  "boaiadaj",
  "boaiaeaa",
  "boaiaeab",
  "boaiaeac",
  "boaiaead",
  "boaiaeae",
  "boaiaeaf",
  "boaiaeag",
  "boaiaeah",
  "boaiaeai",
  "boaiaeaj",
  "boaiafaa",
  "boaiafab",
  "boaiafac",
  "boaiafad",
  "boaiafae",
  "boaiafaf",
  "boaiafag",
  "boaiafah",
  "boaiafai",
  "boaiafaj",
  "boaiagaa",
  "boaiagab",
  "boaiagac",
  "boaiagad",
  "boaiagae",
  "boaiagaf",
  "boaiagag",
  "boaiagah",
  "boaiagai",
  "boaiagaj",
  "boaiahaa",
  "boaiahab",
  "boaiahac",
  "boaiahad",
  "boaiahae",
  "boaiahaf",
  "boaiahag",
  "boaiahah",
  "boaiahai",
  "boaiahaj",
  "boaiaiaa",
  "boaiaiab",
  "boaiaiac",
  "boaiaiad",
  "boaiaiae",
  "boaiaiaf",
  "boaiaiag",
  "boaiaiah",
  "boaiaiai",
  "boaiaiaj",
  "braeahaa",
  "braeahab",
  "braeahac",
  "braeahad",
  "braeahae",
  "braeahaf",
  "braeahag",
  "braeahah",
  "braeahai",
  "braeahaj",
  "braeaiaa",
  "braeaiab",
  "braeaiac",
  "braeaiad",
  "braeaiae",
  "braeaiaf",
  "braeaiag",
  "braeaiah",
  "braeaiai",
  "braeaiaj",
  "brafaeaa",
  "brafaeab",
  "brafaeac",
  "brafaead",
  "brafaeae",
  "brafaeaf",
  "brafaeag",
  "brafaeah",
  "brafaeai",
  "brafaeaj",
  "brafafaa",
  "brafafab",
  "brafafac",
  "brafafad",
  "brafafae",
  "brafafaf",
  "brafafag",
  "brafafah",
  "brafafai",
  "brafafaj",
  "brafagaa",
  "brafagab",
  "brafagac",
  "brafagad",
  "brafagae",
  "brafagaf",
  "brafagag",
  "brafagah",
  "brafagai",
  "brafagaj",
  "brafahaa",
  "brafahab",
  "brafahac",
  "brafahad",
  "brafahae",
  "brafahaf",
  "brafahag",
  "brafahah",
  "brafahai",
  "brafahaj",
  "brafaiaa",
  "brafaiab",
  "brafaiac",
  "brafaiad",
  "brafaiae",
  "brafaiaf",
  "brafaiag",
  "brafaiah",
  "brafaiai",
  "brafaiaj",
  "bxacajaa",
  "bxacajab",
  "bxacajac",
  "bxacajad",
  "bxacajae",
  "bxacajaf",
  "bxacajag",
  "bxacajah",
  "bxacajai",
  "bxacajaj",
  "bxadaaaa",
  "bxadaaab",
  "bxadaaac",
  "bxadaaad",
  "bxadaaae",
  "bxadaaaf",
  "bxadaaag",
  "bxadaaah",
  "bxadaaai",
  "bxadaaaj",
  "bxadabaa",
  "bxadabab",
  "bxadabac",
  "bxadabad",
  "bxadabae",
  "bxadabaf",
  "bxadabag",
  "bxadabah",
  "bxadabai",
  "bxadabaj",
  "cbaiafaa",
  "cbaiafab",
  "cbaiafac",
  "cbaiafad",
  "cbaiafae",
  "cbaiafaf",
  "cbaiafag",
  "cbaiafah",
  "cbaiafai",
  "cbaiafaj",
  "cbaiagaa",
  "cbaiagab",
  "cbaiagac",
  "cbaiagad",
  "cbaiagae",
  "cbaiagaf",
  "cbaiagag",
  "cbaiagah",
  "cbaiagai",
  "cbaiagaj",
  "cbaiahaa",
  "cbaiahab",
  "cbaiahac",
  "cbaiahad",
  "cbaiahae",
  "cbaiahaf",
  "cbaiahag",
  "cbaiahah",
  "cbaiahai",
  "cbaiahaj",
  "cbaiaiaa",
  "cbaiaiab",
  "cbaiaiac",
  "cbaiaiad",
  "cbaiaiae",
  "cbaiaiaf",
  "cbaiaiag",
  "cbaiaiah",
  "cbaiaiai",
  "cbaiaiaj",
  "cbaiajaa",
  "cbaiajab",
  "cbaiajac",
  "cbaiajad",
  "cbaiajae",
  "cbaiajaf",
  "cbaiajag",
  "cbaiajah",
  "cbaiajai",
  "cbaiajaj",
  "ciaiabaa",
  "ciaiabab",
  "ciaiabac",
  "ciaiabad",
  "ciaiabae",
  "ciaiabaf",
  "ciaiabag",
  "ciaiabah",
  "ciaiabai",
  "ciaiabaj",
  "ciaiacaa",
  "ciaiacab",
  "ciaiacac",
  "ciaiacad",
  "ciaiacae",
  "ciaiacaf",
  "ciaiacag",
  "ciaiacah",
  "ciaiacai",
  "ciaiacaj",
  "ciaiaeaa",
  "ciaiaeab",
  "ciaiaeac",
  "ciaiaead",
  "ciaiaeae",
  "ciaiaeaf",
  "ciaiaeag",
  "ciaiaeah",
  "ciaiaeai",
  "ciaiaeaj",
  "ciaiafaa",
  "ciaiafab",
  "ciaiafac",
  "ciaiafad",
  "ciaiafae",
  "ciaiafaf",
  "ciaiafag",
  "ciaiafah",
  "ciaiafai",
  "ciaiafaj",
  "ciaiagaa",
  "ciaiagab",
  "ciaiagac",
  "ciaiagad",
  "ciaiagae",
  "ciaiagaf",
  "ciaiagag",
  "ciaiagah",
  "ciaiagai",
  "ciaiagaj",
  "ciaiahaa",
  "ciaiahab",
  "ciaiahac",
  "ciaiahad",
  "ciaiahae",
  "ciaiahaf",
  "ciaiahag",
  "ciaiahah",
  "ciaiahai",
  "ciaiahaj",
  "ciajaaaa",
  "ciajaaab",
  "ciajaaac",
  "ciajaaad",
  "ciajaaae",
  "ciajaaaf",
  "ciajaaag",
  "ciajaaah",
  "ciajaaai",
  "ciajaaaj",
  "cjabadaa",
  "cjabadab",
  "cjabadac",
  "cjabadad",
  "cjabadae",
  "cjabadaf",
  "cjabadag",
  "cjabadah",
  "cjabadai",
  "cjabadaj",
  "cjabafaa",
  "cjabafab",
  "cjabafac",
  "cjabafad",
  "cjabafae",
  "cjabafaf",
  "cjabafag",
  "cjabafah",
  "cjabafai",
  "cjabafaj",
  "cjabahaa"
]
parts.sort()

# parts processed so far

parts_left = parts.copy()

for part in parts:
    
# keep track of parts left to process in case of error
    
  parts_left.remove(part)

  dataset_cad_filename = dataset_prefix + "REG_dataset_XY-named-hg38-part-" + part +".vcf"
  dataset_cad_df = pd.read_csv(dataset_cad_filename, sep='\t')
  dataset_cad_filename_parquet = dataset_prefix + "REG_dataset_XY-named-hg38-part-" + part +".parquet"
  dataset_cad_df.to_parquet(dataset_cad_filename_parquet)
  dataset_cad = load_dataset_from_file_or_dir(dataset_cad_filename_parquet, split="test", is_file=True)


# create dataset and dataframe

  set_start = 0
  set_end = len(dataset_cad)
  dataset_cad_set = dataset_cad.select(range(set_start, set_end))
  df_cad_set = pd.DataFrame(dataset_cad_set)
  print(df_cad_set)

# tokenize dataset

  dataset_cad_set.set_transform(vep_inf.tokenize_function)

# set arguments for testing

  from transformers import Trainer, TrainingArguments

  batch_size = 500
  training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_eval_batch_size=batch_size,
    dataloader_num_workers=0,
    remove_unused_columns=False,
#   torch_compile=True,
#   fp16=True,
)

  trainer = Trainer(
    model=vep_inf.model,
    args=training_args
  )

# Test and score dataset subset

# for debugging purposes
  !TORCH_LOGS="+dynamo"
  !TORCHDYNAMO_VERBOSE=1

  try:
    pred=trainer.predict(test_dataset=dataset_cad_set).predictions

      
    gpn_score = vep_inf.postprocess(pred)
    df_cad_set["gpn_score"] = gpn_score
    print(df_cad_set["gpn_score"])
        
# drop stub if down to 1-result
        
    df_cad_set.drop([1], axis=0, inplace=True)

    
# Write success to terminal and results file
    
    print(part + ',' + str(df_cad_set["gpn_score"][0]))
    f.write(part + ',' + str(df_cad_set["gpn_score"][0]) + "\n")
    
    
# write score output to file
    
    #output_file = output_dir + "/REG_dataset_XY-named-hg38-part_" + part + "_" + str(set_start) + "_" + 1 + ".csv"
    output_file = output_dir + "/REG_dataset_XY-named-hg38-part_" + part + "_" + str(set_start) + "_" + str(set_end) + ".csv"
    df_cad_set.to_csv(output_file, index=False, sep=',')   
  
  except:
        
# Write failure to terminal and results file
        
    print(part + ",no score") 
    f.write(part + ",no score" + "\n")
    continue
    
# Close results file
    
f.close()


  chrom        pos      name ref alt
0     1  119912963  SNP00241   T   C
1     1  101158974  SNP00201   G   A


0   -0.422637
1    1.683076
Name: gpn_score, dtype: float32
aaacaeaa,-0.4226367
  chrom        pos      name ref alt
0     1  119913940  SNP00242   G   T
1     1  101158974  SNP00201   G   A


0   -0.081638
1    1.683076
Name: gpn_score, dtype: float32
aaacaeab,-0.08163762
  chrom        pos      name ref alt
0     1  119973995  SNP00243   C   T
1     1  101158974  SNP00201   G   A


0   -0.685928
1    1.683076
Name: gpn_score, dtype: float32
aaacaeac,-0.68592787
  chrom        pos      name ref alt
0     1  119987127  SNP00244   A   C
1     1  101158974  SNP00201   G   A


0   -1.574510
1    1.683076
Name: gpn_score, dtype: float32
aaacaead,-1.5745101
  chrom        pos      name ref alt
0     1  145715457  SNP00245   T   C
1     1  101158974  SNP00201   G   A
aaacaeae,no score
  chrom        pos      name ref alt
0     1  145714354  SNP00246   A   T
1     1  101158974  SNP00201   G   A
aaacaeaf,no score
  chrom        pos      name ref alt
0     1  145711946  SNP00247   A   C
1     1  101158974  SNP00201   G   A
aaacaeag,no score
  chrom        pos      name ref alt
0     1  145711421  SNP00248   T   C
1     1  101158974  SNP00201   G   A
aaacaeah,no score
  chrom        pos      name ref alt
0     1  145711327  SNP00249   A   C
1     1  101158974  SNP00201   G   A
aaacaeai,no score
  chrom        pos      name ref alt
0     1  147773279  SNP00250   T   C
1     1  101158974  SNP00201   G   A


...

0   -0.202252
1    1.683076
Name: gpn_score, dtype: float32
cjabafaf,-0.20225167
  chrom        pos      name ref alt
0     X   68949063  SNP61156   T   C
1     1  101158974  SNP00201   G   A


0    0.839906
1    1.683076
Name: gpn_score, dtype: float32
cjabafag,0.83990586
  chrom        pos      name ref alt
0     X  114334931  SNP61157   A   G
1     1  101158974  SNP00201   G   A


0    0.661625
1    1.683076
Name: gpn_score, dtype: float32
cjabafah,0.66162455
  chrom        pos      name ref alt
0     X  115408582  SNP61158   A   G
1     1  101158974  SNP00201   G   A
cjabafai,no score
  chrom        pos      name ref alt
0     X  116004360  SNP61159   C   T
1     1  101158974  SNP00201   G   A


0   -2.076304
1    1.683076
Name: gpn_score, dtype: float32
cjabafaj,-2.0763042
  chrom        pos      name ref alt
0     X  125560120  SNP61170   A   G
1     1  101158974  SNP00201   G   A


0   -0.287936
1    1.683076
Name: gpn_score, dtype: float32
cjabahaa,-0.28793555


Load CAD datasets for P < 0.01 and P > 0.5

In [20]:
output_dir = "/expanse/lustre/projects/nji102/sgriesmer/gpn/output"

cad_gt5_csv_file = output_dir + "/cad-gt5-results.csv"
cad_lt01_csv_file = output_dir + "/cad-lt01-results.csv"
df_cad_set_gt5 = pd.read_csv(cad_gt5_csv_file, sep=',')
df_cad_set_lt01 = pd.read_csv(cad_lt01_csv_file, sep=',')

In [21]:
df_cad_set_gt5

,file suffix,gpn_score
0,aaaaaaaa,no score
1,aaaaaaab,0.89399433
2,aaaaaaac,1.8475147
3,aaaaaaad,-2.8691573
4,aaaaaaae,-1.3439262
...,...,...
9995,ajajajaf,-1.5116944
9996,ajajajag,-1.8452871
9997,ajajajah,no score
9998,ajajajai,no score


In [22]:
df_cad_set_lt01

,file suffix,gpn_score
0,aaaaaaab,0.57608455
1,aaaaaaac,-0.36128122
2,aaaaaaad,no score
3,aaaaaaae,no score
4,aaaaaaaf,no score
...,...,...
9994,ajajajaf,no score
9995,ajajajag,no score
9996,ajajajah,1.2931308
9997,ajajajai,no score


### Drop rows with "No score"

In [17]:
df_cad_set_lt01 = df_cad_set_lt01[df_cad_set_lt01["gpn_score"].str.contains("no score") == False]
df_cad_set_lt01

,file suffix,gpn_score
0,aaaaaaab,0.57608455
1,aaaaaaac,-0.36128122
10,aaaaabab,-0.37237144
11,aaaaabac,-0.8319971
17,aaaaabai,-1.4604263
...,...,...
9991,ajajajac,-3.1450398
9992,ajajajad,0.41094077
9993,ajajajae,0.07117581
9996,ajajajah,1.2931308


In [19]:
df_cad_set_gt5 = df_cad_set_gt5[df_cad_set_gt5["gpn_score"].str.contains("no score") == False]
df_cad_set_gt5

,file suffix,gpn_score
1,aaaaaaac,-0.36128122
10,aaaaabab,-0.37237144
11,aaaaabac,-0.8319971
17,aaaaabai,-1.4604263
20,aaaaacab,0.10453239
...,...,...
9974,ajajahaf,-0.98141
9977,ajajahai,-1.5508695
9987,ajajaiai,-0.33894324
9991,ajajajac,-3.1450398


In [ ]:
part = "afajajag"#  dataset_regulatory_filename = dataset_prefix + "REG_dataset_XY-named-trunc-hg38-part-" + part +".vcf"
dataset_cad_filename = dataset_prefix + "random_sampling_lt01-hg38-part-" + part +".vcf"
dataset_cad = load_dataset_from_file_or_dir(dataset_cad_filename, split="test", is_file=True)
dataset_cad

In [ ]:
set_start = 0
#set_end = len(dataset_cad)
set_end = 2
dataset_cad_set = dataset_cad.select(range(set_start, set_end))
df_cad_set = pd.DataFrame(dataset_cad_set)
df_cad_set, dataset_cad_set

In [ ]:
dataset_cad_set.set_transform(vep_inf.tokenize_function)
dataset_cad_set.features

In [ ]:
from transformers import Trainer, TrainingArguments

output_dir = "/expanse/lustre/projects/nji102/sgriesmer/gpn/output"

batch_size = 500
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_eval_batch_size=batch_size,
    dataloader_num_workers=0,
    remove_unused_columns=False,
#   torch_compile=True,
#   fp16=True,
)


In [ ]:
trainer = Trainer(
    model=vep_inf.model,
    args=training_args
)

In [ ]:
pred=trainer.predict(test_dataset=dataset_cad_set).predictions

In [ ]:
gpn_score = vep_inf.postprocess(pred)
df_cad_set["gpn_score"] = gpn_score

In [ ]:
df_cad_set

### Cut out stub from set

In [ ]:
df_cad_set.drop([1], axis=0, inplace=True)
df_cad_set

In [ ]:
print(part + ',' + str(df_cad_set["gpn_score"][0]))

In [ ]:
output_file = output_dir + "/cad_set_hg38_part_" + part + "_" + str(set_start) + "_" + str(set_end) + ".csv"
df_cad_set.to_csv(output_file, index=False, sep=',') 